In [4]:
# Hawaii Cesspool Matrix Analysis - Master Parcel Attribute Table
# University of Hawaii Water Resources Research Center
# Creates comprehensive parcel attribute table for Matrix sieve analysis

print("=" * 70)
print("Hawaii Cesspool Matrix Analysis")
print("Master Parcel Attribute Table Creation")
print("=" * 70)
print("Academic Framework - University of Hawaii WRRC")

import datetime
import os
import arcpy

print(f"Started: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Set up workspace and environment
arcpy.env.overwriteOutput = True
workspace = arcpy.mp.ArcGISProject("CURRENT").homeFolder
data_dir = os.path.join(workspace, "data", "gis_downloads")
outputs_dir = os.path.join(workspace, "outputs")

print(f"Project workspace: {workspace}")
print(f"Data directory: {data_dir}")
print(f"Outputs directory: {outputs_dir}")

# Create outputs structure
foundation_dir = os.path.join(outputs_dir, "foundation")
if not os.path.exists(foundation_dir):
    os.makedirs(foundation_dir)
    print(f"Created: {foundation_dir}")
else:
    print(f"Using existing: {foundation_dir}")

print("\n" + "=" * 50)
print("STEP 1: VERIFY AVAILABLE DATA")
print("=" * 50)

# Check what layers are available in your map
map_obj = arcpy.mp.ArcGISProject("CURRENT").activeMap
available_layers = [layer.name for layer in map_obj.listLayers()]

print("Available layers in your map:")
for i, layer in enumerate(available_layers, 1):
    print(f"  {i:2d}. {layer}")

print(f"\n✅ Found {len(available_layers)} layers")


Hawaii Cesspool Matrix Analysis
Master Parcel Attribute Table Creation
Academic Framework - University of Hawaii WRRC
Started: 2025-09-25 15:27:52
Project workspace: C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis
Data directory: C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\data\gis_downloads
Outputs directory: C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\outputs
Using existing: C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\outputs\foundation

STEP 1: VERIFY AVAILABLE DATA
Available layers in your map:
   1. World Terrain Reference
   2. World Terrain Base
   3. World Hillshade

✅ Found 3 layers


In [5]:
print("\n" + "=" * 50)
print("STEP 2: LOCATE DATA FILES")
print("=" * 50)

# Search for well distance data files
wells_dir = os.path.join(data_dir, "wells")
print(f"Searching in: {wells_dir}")

if os.path.exists(wells_dir):
    print("✅ Wells directory found")
    
    # Find shapefiles in wells directory
    wells_files = []
    for root, dirs, files in os.walk(wells_dir):
        for file in files:
            if file.endswith('.shp'):
                wells_files.append(os.path.join(root, file))
    
    print(f"Found {len(wells_files)} shapefiles in wells directory:")
    for i, file in enumerate(wells_files, 1):
        filename = os.path.basename(file)
        print(f"  {i}. {filename}")
        
        # Try to identify municipal vs domestic
        if 'municipal' in filename.lower() or 'public' in filename.lower():
            print(f"     → Likely MUNICIPAL wells data")
        elif 'domestic' in filename.lower() or 'private' in filename.lower():
            print(f"     → Likely DOMESTIC wells data")
else:
    print("⚠️ Wells directory not found, checking other locations...")

# Search for soils data
soils_dir = os.path.join(data_dir, "soils")
print(f"\nSearching for soils in: {soils_dir}")

if os.path.exists(soils_dir):
    print("✅ Soils directory found")
    
    soils_files = []
    for root, dirs, files in os.walk(soils_dir):
        for file in files:
            if file.endswith('.shp'):
                soils_files.append(os.path.join(root, file))
    
    print(f"Found {len(soils_files)} soil shapefiles:")
    for i, file in enumerate(soils_files, 1):
        filename = os.path.basename(file)
        print(f"  {i}. {filename}")
else:
    print("⚠️ Soils directory not found")

# Also check if we have any geodatabase files
print(f"\nChecking for geodatabases in project...")
gdb_files = []
for root, dirs, files in os.walk(workspace):
    for dir_name in dirs:
        if dir_name.endswith('.gdb'):
            gdb_path = os.path.join(root, dir_name)
            gdb_files.append(gdb_path)

if gdb_files:
    print(f"Found {len(gdb_files)} geodatabases:")
    for i, gdb in enumerate(gdb_files, 1):
        gdb_name = os.path.basename(gdb)
        print(f"  {i}. {gdb_name}")
        
        # List feature classes in the geodatabase
        arcpy.env.workspace = gdb
        feature_classes = arcpy.ListFeatureClasses()
        if feature_classes:
            print(f"     Contains {len(feature_classes)} feature classes:")
            for fc in feature_classes[:5]:  # Show first 5
                print(f"       - {fc}")
            if len(feature_classes) > 5:
                print(f"       ... and {len(feature_classes) - 5} more")
        else:
            print("     No feature classes found")

print(f"\n🎯 Data inventory complete - ready to identify our key files!")


STEP 2: LOCATE DATA FILES
Searching in: C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\data\gis_downloads\wells
✅ Wells directory found
Found 2 shapefiles in wells directory:
  1. CPs_Distance_to_Domestic_Wells.shp
     → Likely DOMESTIC wells data
  2. CPs_Distance_to_Municipal_Wells.shp
     → Likely MUNICIPAL wells data

Searching for soils in: C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\data\gis_downloads\soils
✅ Soils directory found
Found 2 soil shapefiles:
  1. CPs_Soil_Suitability_Rank.shp
  2. HIstate_nrcs_join2.shp

Checking for geodatabases in project...
Found 2 geodatabases:
  1. ParcelAnalysis.gdb
     Contains 5 feature classes:
       - Parcels_Hawaii
       - tmk_state
       - Parcels_Honolulu
       - Parcels_Kauai
       - Parcels_Maui
  2. mlra_a_hi.gdb
     No feature classes found

🎯 Data inventory complete - ready to identify our key files!


In [7]:
print("\n" + "=" * 50)
print("STEP 3: VERIFY FILE PATHS AND EXAMINE DATA")
print("=" * 50)

# First, let's verify the exact file paths
wells_dir = os.path.join(data_dir, "wells")
soils_dir = os.path.join(data_dir, "soils")

print("Verifying file paths:")
print(f"Wells directory: {wells_dir}")
print(f"Soils directory: {soils_dir}")

# List actual files to confirm exact names
print(f"\nFiles in wells directory:")
if os.path.exists(wells_dir):
    wells_files = [f for f in os.listdir(wells_dir) if f.endswith('.shp')]
    for file in wells_files:
        full_path = os.path.join(wells_dir, file)
        file_exists = os.path.exists(full_path)
        print(f"  {file} - Exists: {file_exists}")
        if file_exists:
            print(f"    Full path: {full_path}")
else:
    print("  Directory not found!")

print(f"\nFiles in soils directory:")
if os.path.exists(soils_dir):
    soils_files = [f for f in os.listdir(soils_dir) if f.endswith('.shp')]
    for file in soils_files:
        full_path = os.path.join(soils_dir, file)
        file_exists = os.path.exists(full_path)
        print(f"  {file} - Exists: {file_exists}")
        if file_exists:
            print(f"    Full path: {full_path}")
else:
    print("  Directory not found!")

# Try to access the first wells file we find
if os.path.exists(wells_dir):
    wells_shapefiles = [f for f in os.listdir(wells_dir) if f.endswith('.shp')]
    
    if wells_shapefiles:
        # Try the municipal wells file
        municipal_wells_file = None
        for file in wells_shapefiles:
            if 'municipal' in file.lower():
                municipal_wells_file = os.path.join(wells_dir, file)
                break
        
        if municipal_wells_file and os.path.exists(municipal_wells_file):
            print(f"\n📊 Testing file access: {os.path.basename(municipal_wells_file)}")
            try:
                # Test if ArcPy can read the file
                test_count = int(arcpy.management.GetCount(municipal_wells_file)[0])
                print(f"✅ Success! Records: {test_count:,}")
                
                # Get field info
                fields = arcpy.ListFields(municipal_wells_file)
                field_names = [f.name for f in fields]
                print(f"✅ Fields ({len(field_names)}): {field_names}")
                
            except Exception as e:
                print(f"❌ Error accessing file: {str(e)}")
        else:
            print("❌ Municipal wells file not found or inaccessible")
    else:
        print("❌ No shapefile found in wells directory")

print(f"\n🔧 Troubleshooting complete - identifying the issue...")


STEP 3: VERIFY FILE PATHS AND EXAMINE DATA
Verifying file paths:
Wells directory: C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\data\gis_downloads\wells
Soils directory: C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\data\gis_downloads\soils

Files in wells directory:

Files in soils directory:
❌ No shapefile found in wells directory

🔧 Troubleshooting complete - identifying the issue...


In [9]:
print("\n" + "=" * 50)
print("STEP 3: COMPREHENSIVE DATA SEARCH")
print("=" * 50)

def find_data_files(base_directory, file_patterns=None, file_types=None):
    """
    Comprehensive recursive search for data files
    Returns organized dictionary of found files by category
    """
    if file_patterns is None:
        file_patterns = ['municipal', 'domestic', 'soil', 'nrcs', 'wells', 'distance']
    if file_types is None:
        file_types = ['.shp', '.gdb']
    
    found_files = {
        'municipal_wells': [],
        'domestic_wells': [], 
        'soils_nrcs': [],
        'soils_hcpt': [],
        'other': []
    }
    
    if not os.path.exists(base_directory):
        return found_files
    
    print(f"🔍 Searching recursively in: {base_directory}")
    
    # Search through all subdirectories
    for root, dirs, files in os.walk(base_directory):
        # Skip hidden directories
        dirs[:] = [d for d in dirs if not d.startswith('.')]
        
        for file in files:
            if any(file.lower().endswith(ext) for ext in file_types):
                full_path = os.path.join(root, file)
                relative_path = os.path.relpath(full_path, base_directory)
                
                # Categorize files
                filename_lower = file.lower()
                
                if 'municipal' in filename_lower and 'well' in filename_lower:
                    found_files['municipal_wells'].append((file, full_path, relative_path))
                elif 'domestic' in filename_lower and 'well' in filename_lower:
                    found_files['domestic_wells'].append((file, full_path, relative_path))
                elif 'nrcs' in filename_lower or 'histate' in filename_lower:
                    found_files['soils_nrcs'].append((file, full_path, relative_path))
                elif 'soil' in filename_lower and 'suitability' in filename_lower:
                    found_files['soils_hcpt'].append((file, full_path, relative_path))
                elif any(pattern in filename_lower for pattern in file_patterns):
                    found_files['other'].append((file, full_path, relative_path))
    
    return found_files

# Search all data directories
print("=" * 60)
print("COMPREHENSIVE DATA INVENTORY")
print("=" * 60)

# Search main data directory
all_data = find_data_files(data_dir)

# Display results by category
categories = {
    'municipal_wells': '🏛️ MUNICIPAL WELLS DATA',
    'domestic_wells': '🏠 DOMESTIC WELLS DATA', 
    'soils_nrcs': '🌱 NRCS SOILS DATA',
    'soils_hcpt': '📊 HCPT SOILS DATA',
    'other': '📁 OTHER RELEVANT FILES'
}

key_files = {}

for category, files in all_data.items():
    if files:
        print(f"\n{categories[category]}:")
        print("-" * 40)
        for i, (filename, full_path, rel_path) in enumerate(files, 1):
            print(f"  {i}. {filename}")
            print(f"     📂 {rel_path}")
            
            # Store first file of each category as key file
            if category not in key_files:
                key_files[category] = full_path
                print(f"     ⭐ Selected as primary {category.replace('_', ' ')} file")

# Test access to key files
print(f"\n" + "=" * 60)
print("TESTING DATA ACCESS")
print("=" * 60)

for category, file_path in key_files.items():
    if category in ['municipal_wells', 'domestic_wells', 'soils_nrcs']:
        print(f"\n📊 Testing {category.replace('_', ' ').title()}: {os.path.basename(file_path)}")
        
        try:
            # Test basic access
            if arcpy.Exists(file_path):
                record_count = int(arcpy.management.GetCount(file_path)[0])
                fields = [f.name for f in arcpy.ListFields(file_path)]
                
                print(f"   ✅ Records: {record_count:,}")
                print(f"   ✅ Fields: {len(fields)} total")
                
                # Show key fields
                key_field_patterns = ['tmk', 'dist', 'ksat', 'slope', 'perc', 'island']
                key_fields = [f for f in fields if any(pattern in f.lower() for pattern in key_field_patterns)]
                if key_fields:
                    print(f"   🔑 Key fields: {key_fields}")
                
            else:
                print(f"   ❌ File not accessible to ArcPy")
                
        except Exception as e:
            print(f"   ❌ Error: {str(e)}")

print(f"\n🎯 DATA INVENTORY COMPLETE!")
print(f"Ready to create master attribute table with recursive file detection.")

# Store key file paths for next steps
if 'municipal_wells' in key_files:
    municipal_wells_path = key_files['municipal_wells']
    print(f"\n📋 KEY FILES IDENTIFIED:")
    print(f"   Municipal Wells: {municipal_wells_path}")
    
if 'domestic_wells' in key_files:
    domestic_wells_path = key_files['domestic_wells'] 
    print(f"   Domestic Wells: {domestic_wells_path}")
    
if 'soils_nrcs' in key_files:
    soils_path = key_files['soils_nrcs']
    print(f"   NRCS Soils: {soils_path}")


STEP 3: COMPREHENSIVE DATA SEARCH
COMPREHENSIVE DATA INVENTORY
🔍 Searching recursively in: C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\data\gis_downloads

🏛️ MUNICIPAL WELLS DATA:
----------------------------------------
  1. CPs_Distance_to_Municipal_Wells.shp
     📂 wells\statewide\CPs_Distance_to_Municipal_Wells.shp
     ⭐ Selected as primary municipal wells file

🏠 DOMESTIC WELLS DATA:
----------------------------------------
  1. CPs_Distance_to_Domestic_Wells.shp
     📂 wells\statewide\CPs_Distance_to_Domestic_Wells.shp
     ⭐ Selected as primary domestic wells file

🌱 NRCS SOILS DATA:
----------------------------------------
  1. HIstate_nrcs_join2.shp
     📂 soils\Statewide\HIstate_nrcs_join2\HIstate_nrcs_join2.shp
     ⭐ Selected as primary soils nrcs file

📊 HCPT SOILS DATA:
----------------------------------------
  1. CPs_Soil_Suitability_Rank.shp
     📂 soils\Statewide\CPs_Soil_Suitablity_Rank\CPs_Soil_Suitability_Rank.shp
     ⭐ Selected as primary soil

In [11]:
print("\n" + "=" * 50)
print("STEP 4: CREATE MASTER ATTRIBUTE TABLE (FIXED)")
print("=" * 50)

# Define our verified file paths
municipal_wells_path = r"C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\data\gis_downloads\wells\statewide\CPs_Distance_to_Municipal_Wells.shp"
domestic_wells_path = r"C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\data\gis_downloads\wells\statewide\CPs_Distance_to_Domestic_Wells.shp" 
soils_path = r"C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\data\gis_downloads\soils\Statewide\HIstate_nrcs_join2\HIstate_nrcs_join2.shp"

print("Creating master attribute table from municipal wells foundation...")

# Create master attribute table
master_table_path = os.path.join(foundation_dir, "TMK_Master_Attributes.shp")

# Copy municipal wells as foundation (82,141 TMK records)
arcpy.management.CopyFeatures(municipal_wells_path, master_table_path)

record_count = int(arcpy.management.GetCount(master_table_path)[0])
print(f"Foundation created: {record_count:,} TMK records")
print(f"Location: {master_table_path}")

# Examine current fields
current_fields = [f.name for f in arcpy.ListFields(master_table_path)]
print(f"Current fields: {current_fields}")

# Add domestic wells distance
print(f"\nJoining domestic wells distance...")

# Join domestic wells data
arcpy.management.JoinField(
    in_data=master_table_path,
    in_field="TMK",  
    join_table=domestic_wells_path,
    join_field="TMK",
    fields=["dist2_DomW"]
)

print(f"Domestic wells distance joined")

# Verify the join worked
updated_fields = [f.name for f in arcpy.ListFields(master_table_path)]
new_fields = [f for f in updated_fields if f not in current_fields]
print(f"New fields added: {new_fields}")

# Add Matrix fields with shorter names (shapefiles have 10-char limit)
print(f"\nAdding Matrix criteria fields...")

matrix_fields = [
    ("SLOPE_PCT", "DOUBLE", "Site slope percentage"),
    ("SOIL_PERC", "DOUBLE", "Soil percolation rate"),  
    ("SOIL_KSAT", "DOUBLE", "Soil hydraulic conductivity"),
    ("AVAIL_AREA", "DOUBLE", "Available land area"),
    ("ZONING", "TEXT", "Zoning classification"),
    ("SEPTIC_OK", "SHORT", "Conventional septic suitable"),
    ("ATU_OK", "SHORT", "ATU system suitable"), 
    ("SEEPAGE_OK", "SHORT", "Seepage pit suitable"),
    ("PROC_LOG", "TEXT", "Processing steps log")  # Shortened name
]

successful_fields = []
for field_name, field_type, field_alias in matrix_fields:
    try:
        if field_type == "TEXT":
            arcpy.management.AddField(master_table_path, field_name, field_type, 
                                   field_alias=field_alias, field_length=50)
        else:
            arcpy.management.AddField(master_table_path, field_name, field_type, 
                                   field_alias=field_alias)
        print(f"   Added: {field_name}")
        successful_fields.append(field_name)
    except Exception as e:
        print(f"   Warning {field_name}: {str(e)}")

print(f"Successfully added {len(successful_fields)} Matrix fields")

# Check what fields actually exist now
final_fields = [f.name for f in arcpy.ListFields(master_table_path)]
print(f"All fields in table: {final_fields}")

# Initialize processing log only if PROC_LOG field exists
if "PROC_LOG" in final_fields:
    print(f"\nInitializing processing log...")
    timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M')
    
    with arcpy.da.UpdateCursor(master_table_path, ["PROC_LOG"]) as cursor:
        count = 0
        for row in cursor:
            row[0] = f"Created: {timestamp}"
            cursor.updateRow(row)
            count += 1
            if count % 20000 == 0:
                print(f"   Processed {count:,} records...")
    
    print(f"Processing log initialized for {count:,} records")
else:
    print(f"PROC_LOG field not available - skipping log initialization")

# Sample data check
print(f"\nSample data (first 3 records):")
sample_fields = ["TMK", "Island", "dist2_MunW", "dist2_DomW"]
sample_fields = [f for f in sample_fields if f in final_fields]

with arcpy.da.SearchCursor(master_table_path, sample_fields) as cursor:
    for i, row in enumerate(cursor):
        if i < 3:
            record_data = []
            for j, value in enumerate(row):
                record_data.append(f"{sample_fields[j]}: {value}")
            print(f"  Record {i+1}: {' | '.join(record_data)}")
        if i >= 2:
            break

print(f"\nMASTER ATTRIBUTE TABLE COMPLETE!")
print(f"   Records: {record_count:,}")
print(f"   Total fields: {len(final_fields)}")
print(f"   Ready for soil data processing")


STEP 4: CREATE MASTER ATTRIBUTE TABLE (FIXED)
Creating master attribute table from municipal wells foundation...
Foundation created: 82,141 TMK records
Location: C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\outputs\foundation\TMK_Master_Attributes.shp
Current fields: ['FID', 'Shape', 'X', 'Y', 'Island', 'TMK', 'Uid', 'dist2_MunW']

Joining domestic wells distance...
Domestic wells distance joined
New fields added: ['dist2_DomW']

Adding Matrix criteria fields...
   Added: SLOPE_PCT
   Added: SOIL_PERC
   Added: SOIL_KSAT
   Added: AVAIL_AREA
   Added: ZONING
   Added: SEPTIC_OK
   Added: ATU_OK
   Added: SEEPAGE_OK
   Added: PROC_LOG
Successfully added 9 Matrix fields
All fields in table: ['FID', 'Shape', 'X', 'Y', 'Island', 'TMK', 'Uid', 'dist2_MunW', 'dist2_DomW', 'SLOPE_PCT', 'SOIL_PERC', 'SOIL_KSAT', 'AVAIL_AREA', 'ZONING', 'SEPTIC_OK', 'ATU_OK', 'SEEPAGE_OK', 'PROC_LOG']

Initializing processing log...
   Processed 20,000 records...
   Processed 40,000 records...

In [12]:
print("\n" + "=" * 70)
print("SOIL CLASSIFICATION METHODOLOGY REVIEW AND DOCUMENTATION")
print("=" * 70)

# First, let's examine what we actually have in the NRCS data
soils_path = r"C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\data\gis_downloads\soils\Statewide\HIstate_nrcs_join2\HIstate_nrcs_join2.shp"

print("EXAMINING ACTUAL NRCS SOIL DATA:")
print("-" * 40)

# Get all fields and their properties
soil_fields = arcpy.ListFields(soils_path)
print(f"Total fields in NRCS data: {len(soil_fields)}")

# Focus on hydraulic conductivity fields
ksat_fields = [f for f in soil_fields if 'ksat' in f.name.lower()]
print(f"\nHydraulic conductivity fields found:")
for field in ksat_fields:
    print(f"  {field.name}: {field.type}, Length: {field.length}")

# Sample actual KSAT values to understand the data distribution
print(f"\nSample KSAT values (first 20 records):")
ksat_field_names = [f.name for f in ksat_fields]
if ksat_field_names:
    with arcpy.da.SearchCursor(soils_path, ksat_field_names) as cursor:
        for i, row in enumerate(cursor):
            if i < 20:
                values = [f"{ksat_field_names[j]}: {row[j]}" for j in range(len(row))]
                print(f"  Record {i+1}: {' | '.join(values)}")
            if i >= 19:
                break
else:
    print("  No KSAT fields found")

# Check for other relevant fields
relevant_patterns = ['slope', 'drain', 'perc', 'hydro', 'restrict']
relevant_fields = []
for field in soil_fields:
    if any(pattern in field.name.lower() for pattern in relevant_patterns):
        relevant_fields.append(field)

print(f"\nOther potentially relevant fields:")
for field in relevant_fields[:10]:  # Show first 10
    print(f"  {field.name}: {field.type}")

print(f"\n" + "=" * 70)
print("METHODOLOGY ISSUES TO ADDRESS:")
print("=" * 70)
print("1. KSAT to Percolation Rate Conversion:")
print("   - KSAT units need verification (micrometers/sec vs other)")
print("   - Conversion formula needs literature backing")
print("   - HAR 11-62 requires specific percolation rate ranges")
print("   - Current conversion is approximate, not scientifically rigorous")
print("")
print("2. Missing Critical Data:")
print("   - No direct percolation test data")
print("   - Drainage class information unclear") 
print("   - Depth to restrictive layer not identified")
print("")
print("3. Regulatory Compliance:")
print("   - HAR 11-62 requires site-specific percolation testing")
print("   - Soil survey data is reconnaissance level, not design level")
print("   - Matrix analysis should note limitations")


SOIL CLASSIFICATION METHODOLOGY REVIEW AND DOCUMENTATION
EXAMINING ACTUAL NRCS SOIL DATA:
----------------------------------------
Total fields in NRCS data: 22

Hydraulic conductivity fields found:
  ksat_h: String, Length: 254
  ksat_l: String, Length: 254
  ksat_r: String, Length: 254

Sample KSAT values (first 20 records):
  Record 1: ksat_h: 16.46 | ksat_l: 3.5283333333333338 | ksat_r: 9.866666666666667
  Record 2: ksat_h: 15.88 | ksat_l: 1.5533333333333335 | ksat_r: 8.666666666666666
  Record 3: ksat_h: NoData | ksat_l: NoData | ksat_r: NoData
  Record 4: ksat_h: 77.57 | ksat_l: 9.17 | ksat_r: 39.5
  Record 5: ksat_h: 4.946666666666667 | ksat_l: 1.3466666666666667 | ksat_r: 3.1466666666666665
  Record 6: ksat_h: 18.5575 | ksat_l: 2.115 | ksat_r: 9.875
  Record 7: ksat_h: 32.1 | ksat_l: 3.227619047619048 | ksat_r: 17.576190476190476
  Record 8: ksat_h: 77.57 | ksat_l: 9.17 | ksat_r: 39.5
  Record 9: ksat_h: NoData | ksat_l: NoData | ksat_r: NoData
  Record 10: ksat_h: 141.0 | ksa

In [13]:
print("\n" + "=" * 50)
print("STEP 6: UPDATE EXISTING TECHNOLOGY MATRIX")
print("=" * 50)

import pandas as pd

# Path to existing Matrix file
matrix_path = r"C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\Matrix\Technology,Treatment,Disposal,Treat.xls"

print(f"Found existing Matrix file: {matrix_path}")

# Read the existing Matrix
try:
    # Try reading the Excel file
    df_matrix = pd.read_excel(matrix_path)
    print(f"Matrix loaded successfully: {len(df_matrix)} technologies, {len(df_matrix.columns)} criteria")
    
    # Display basic structure
    print(f"\nCurrent Matrix structure:")
    print(f"Columns: {list(df_matrix.columns)}")
    
    if 'Technology' in df_matrix.columns:
        print(f"\nTechnologies in Matrix:")
        for i, tech in enumerate(df_matrix['Technology'], 1):
            print(f"  {i:2d}. {tech}")
    
    # Look for existing soil-related columns
    soil_columns = [col for col in df_matrix.columns if any(keyword in col.lower() for keyword in ['soil', 'perc', 'infiltra', 'ksat'])]
    print(f"\nExisting soil-related columns: {soil_columns}")
    
    # Check current matrix documentation folder
    matrix_docs_path = r"C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\Matrix\documentation"
    if not os.path.exists(matrix_docs_path):
        os.makedirs(matrix_docs_path)
        print(f"Created documentation folder: {matrix_docs_path}")
    
    # Create updated Matrix with screening disclaimers
    print(f"\nUpdating Matrix with screening-level approach...")
    
    # Add disclaimer columns if they don't exist
    disclaimer_columns = {
        'SCREENING_TOOL_ONLY': 'This Matrix provides preliminary screening only - professional engineering required',
        'SITE_TESTING_REQD': 'Site-specific testing and evaluation required for all installations',
        'DOH_PERMIT_REQD': 'DOH Individual Wastewater System permit required'
    }
    
    df_updated = df_matrix.copy()
    
    # Add disclaimer columns
    for col_name, col_value in disclaimer_columns.items():
        if col_name not in df_updated.columns:
            df_updated[col_name] = col_value
            print(f"  Added disclaimer column: {col_name}")
    
    # Update soil-related columns with screening classifications
    # Replace any existing soil columns with new screening approach
    if soil_columns:
        print(f"Updating existing soil columns to screening classifications...")
        for col in soil_columns:
            print(f"  Updating column: {col}")
        
        # For now, create new screening-based columns
        soil_screening_columns = {
            'High_Infiltration_Potential': 'KSAT >25 μm/s - Screening indicates favorable conditions',
            'Moderate_Infiltration_Potential': 'KSAT 5-25 μm/s - Site testing critical',
            'Low_Infiltration_Potential': 'KSAT <5 μm/s - Consider alternatives', 
            'Data_Insufficient': 'No soil survey data - Site investigation required'
        }
        
        for col_name, description in soil_screening_columns.items():
            df_updated[col_name] = 0  # Default to not suitable, requires case-by-case review
            print(f"  Added screening column: {col_name}")
    
    # Save updated Matrix
    updated_matrix_path = os.path.join(os.path.dirname(matrix_path), "Technology_Matrix_Screening_Updated.xlsx")
    df_updated.to_excel(updated_matrix_path, index=False)
    print(f"Updated Matrix saved: {updated_matrix_path}")
    
    # Create change log
    change_log = f"""
TECHNOLOGY MATRIX UPDATE LOG
===========================
Date: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
Original File: {matrix_path}
Updated File: {updated_matrix_path}

CHANGES MADE:
============

1. ADDED DISCLAIMER COLUMNS:
   - SCREENING_TOOL_ONLY: Emphasizes preliminary screening purpose
   - SITE_TESTING_REQD: Requires site-specific evaluation
   - DOH_PERMIT_REQD: Mandates regulatory compliance

2. UPDATED SOIL CLASSIFICATION APPROACH:
   - Replaced site-specific soil requirements with screening-level classifications
   - Added four screening categories based on NRCS KSAT data
   - Conservative approach requiring professional follow-up

3. SCREENING-LEVEL CLASSIFICATIONS ADDED:
   - High_Infiltration_Potential: KSAT >25 μm/s from soil survey data
   - Moderate_Infiltration_Potential: KSAT 5-25 μm/s from soil survey data  
   - Low_Infiltration_Potential: KSAT <5 μm/s from soil survey data
   - Data_Insufficient: No reliable soil survey data available

4. CONSERVATIVE DEFAULT VALUES:
   - All screening columns default to 0 (not suitable)
   - Requires case-by-case professional evaluation
   - Emphasizes need for site-specific testing

RATIONALE:
=========
- Matrix now serves as screening tool for planning purposes only
- Removes inappropriate site-specific design elements
- Adds extensive disclaimers about limitations
- Aligns with professional engineering requirements
- Meets academic standards for uncertainty acknowledgment

NEXT STEPS:
==========
1. Review updated Matrix with subject matter experts
2. Populate screening columns with appropriate technology compatibility
3. Test screening approach with pilot parcels
4. Document validation and verification procedures
5. Integrate with master attribute table workflow
"""
    
    change_log_path = os.path.join(matrix_docs_path, "Matrix_Update_Change_Log.txt")
    with open(change_log_path, 'w', encoding='utf-8') as f:
        f.write(change_log)
    
    print(f"Change log created: {change_log_path}")
    
    print(f"\n" + "=" * 50)
    print("MATRIX UPDATE COMPLETE")
    print("=" * 50)
    print("Updated Matrix emphasizes screening purpose with:")
    print("- Disclaimer columns for all technologies")
    print("- Screening-level soil classifications")  
    print("- Conservative default requirements")
    print("- Extensive documentation of limitations")
    print("- Professional engineering requirements")

except Exception as e:
    print(f"Error reading Matrix file: {str(e)}")
    print("Please verify the file exists and is accessible")


STEP 6: UPDATE EXISTING TECHNOLOGY MATRIX
Found existing Matrix file: C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\Matrix\Technology,Treatment,Disposal,Treat.xls
Matrix loaded successfully: 24 technologies, 42 criteria

Current Matrix structure:
Columns: ['Technology', 'Treatment', 'Disposal', 'Treatment & Disposal', 'DOH Approved', 'DOH Conditional Approval', 'DOH Experimental/Limited', 'Depth <3 ft', 'Depth 3-6 ft', 'Depth >6 ft', 'Lot <10k sf', 'Lot 10k-21k sf', 'Lot >21k sf', 'Slope <8%', 'Slope 8-12%', 'Slope >12%', 'SMA Within 50 ft', 'SMA Beyond 50 ft', 'SMA No', 'Soil <1 min/inch', 'Soil 1-10 min/inch', 'Soil 10-60 min/inch', 'Soil >60 min/inch', 'Soil N/A', 'N Removal High', 'N Removal Medium', 'N Removal Low/None', 'N Removal N/A', 'Stream <50 ft Yes', 'Stream <50 ft No', 'Stream <50 ft Conditional', 'Well 100 ft', 'Well 1000 ft', 'Well Variable', 'Well No Setback', 'Climate Xeric', 'Climate Mesic', 'Climate Hydric/Intense Rainfall', 'Flood Zone Yes', 'Flood

In [14]:
print("\n" + "=" * 50)
print("STEP 7: CLEAN UP NUMERIC FIELDS - REMOVE DECIMALS")
print("=" * 50)

# Path to master attribute table
master_table_path = os.path.join(foundation_dir, "TMK_Master_Attributes.shp")

print(f"Cleaning numeric fields in: {os.path.basename(master_table_path)}")

# Get all numeric fields that need cleaning
fields = arcpy.ListFields(master_table_path)
numeric_fields_to_clean = []

for field in fields:
    if field.type in ['Double', 'Single'] and field.name not in ['Shape_Length', 'Shape_Area']:
        numeric_fields_to_clean.append(field.name)
        print(f"  Will clean: {field.name} ({field.type})")

print(f"\nFound {len(numeric_fields_to_clean)} numeric fields to clean")

if numeric_fields_to_clean:
    print("Rounding to whole numbers...")
    
    # Process in chunks for better performance
    chunk_size = 10000
    total_records = int(arcpy.management.GetCount(master_table_path)[0])
    processed = 0
    
    with arcpy.da.UpdateCursor(master_table_path, numeric_fields_to_clean) as cursor:
        for row in cursor:
            # Round each numeric value to whole number
            cleaned_row = []
            for value in row:
                if value is not None:
                    cleaned_row.append(round(float(value)))
                else:
                    cleaned_row.append(value)
            
            cursor.updateRow(cleaned_row)
            processed += 1
            
            # Progress indicator
            if processed % chunk_size == 0:
                print(f"    Processed {processed:,} of {total_records:,} records...")
    
    print(f"Completed cleaning {processed:,} records")
    
    # Verify the changes
    print("\nSample of cleaned data (first 3 records):")
    sample_fields = ['TMK', 'dist2_MunW', 'dist2_DomW'] + numeric_fields_to_clean[:2]
    sample_fields = [f for f in sample_fields if f in [field.name for field in arcpy.ListFields(master_table_path)]]
    
    with arcpy.da.SearchCursor(master_table_path, sample_fields) as cursor:
        for i, row in enumerate(cursor):
            if i < 3:
                record_data = []
                for j, value in enumerate(row):
                    record_data.append(f"{sample_fields[j]}: {value}")
                print(f"  Record {i+1}: {' | '.join(record_data)}")
            if i >= 2:
                break

else:
    print("No numeric fields found to clean")

print(f"\n" + "=" * 50)
print("NUMERIC FIELD CLEANUP COMPLETE")
print("=" * 50)
print("All distance and numeric fields now show as whole numbers")
print("Data is cleaner and more professional for 82,000+ records")


STEP 7: CLEAN UP NUMERIC FIELDS - REMOVE DECIMALS
Cleaning numeric fields in: TMK_Master_Attributes.shp
  Will clean: X (Double)
  Will clean: Y (Double)
  Will clean: dist2_MunW (Double)
  Will clean: dist2_DomW (Double)
  Will clean: SLOPE_PCT (Double)
  Will clean: SOIL_PERC (Double)
  Will clean: SOIL_KSAT (Double)
  Will clean: AVAIL_AREA (Double)

Found 8 numeric fields to clean
Rounding to whole numbers...
    Processed 10,000 of 82,141 records...
    Processed 20,000 of 82,141 records...
    Processed 30,000 of 82,141 records...
    Processed 40,000 of 82,141 records...
    Processed 50,000 of 82,141 records...
    Processed 60,000 of 82,141 records...
    Processed 70,000 of 82,141 records...
    Processed 80,000 of 82,141 records...
Completed cleaning 82,141 records

Sample of cleaned data (first 3 records):
  Record 1: TMK: 186006001 | dist2_MunW: 3180.0 | dist2_DomW: 2913.0 | X: -158.0 | Y: 21.0
  Record 2: TMK: 186006001 | dist2_MunW: 3203.0 | dist2_DomW: 2913.0 | X: -158

In [18]:
print("\n" + "=" * 70)
print("CLEANUP BAD OUTPUTS AND EXAMINE DATA STRUCTURES")
print("=" * 70)

# Clean up the problematic files we created
cleanup_files = [
    os.path.join(foundation_dir, "Cesspool_Parcel_Polygons.shp"),
    os.path.join(foundation_dir, "Maui_Cesspool_Polygons.shp"),
    os.path.join(foundation_dir, "temp_parcels.shp"),
    os.path.join(foundation_dir, "cesspool_tmks.dbf"),
    os.path.join(foundation_dir, "Cesspool_Parcels_Statewide.shp"),
    os.path.join(foundation_dir, "Maui_Cesspool_Points.shp"),
    os.path.join(foundation_dir, "Maui_Cesspool_Parcels.shp")
]

print("Cleaning up problematic output files...")
for file_path in cleanup_files:
    try:
        if arcpy.Exists(file_path):
            arcpy.management.Delete(file_path)
            print(f"  Deleted: {os.path.basename(file_path)}")
    except Exception as e:
        print(f"  Could not delete {os.path.basename(file_path)}: {str(e)}")

# Also remove any related files (.dbf, .shx, .prj, etc.)
for cleanup_file in cleanup_files:
    if cleanup_file.endswith('.shp'):
        base_name = cleanup_file[:-4]
        for ext in ['.dbf', '.shx', '.prj', '.cpg', '.sbn', '.sbx', '.xml']:
            try:
                related_file = base_name + ext
                if os.path.exists(related_file):
                    os.remove(related_file)
            except:
                pass

print("✓ Cleanup complete")

print(f"\nEXAMINING DATA STRUCTURE COMPATIBILITY")
print("=" * 50)

# Examine cesspool points (master attribute table)
master_table_path = os.path.join(foundation_dir, "TMK_Master_Attributes.shp")
print("CESSPOOL POINTS DATA (TMK_Master_Attributes.shp):")

if arcpy.Exists(master_table_path):
    cesspool_count = int(arcpy.management.GetCount(master_table_path)[0])
    cesspool_fields = [(f.name, f.type, f.length) for f in arcpy.ListFields(master_table_path)]
    
    print(f"  Records: {cesspool_count:,}")
    print(f"  TMK field details:")
    
    tmk_field_info = next((f for f in cesspool_fields if f[0] == 'TMK'), None)
    if tmk_field_info:
        print(f"    Name: {tmk_field_info[0]}")
        print(f"    Type: {tmk_field_info[1]}")
        print(f"    Length: {tmk_field_info[2]}")
    
    # Sample TMK values
    print("  Sample TMK values:")
    with arcpy.da.SearchCursor(master_table_path, ['TMK', 'Island']) as cursor:
        for i, row in enumerate(cursor):
            if i < 8:
                print(f"    {row[0]} ({row[1]})")
            if i >= 7:
                break

# Examine parcel polygons
tmk_parcels_path = None
possible_paths = [
    os.path.join(workspace, "data", "tmk_state.shp"),
    os.path.join(workspace, "data", "tmk_state.shp", "tmk_state.shp")
]

for path in possible_paths:
    if arcpy.Exists(path):
        tmk_parcels_path = path
        break

print(f"\nTMK PARCEL POLYGONS DATA:")

if tmk_parcels_path:
    print(f"  Found at: {tmk_parcels_path}")
    
    try:
        parcel_count = int(arcpy.management.GetCount(tmk_parcels_path)[0])
        parcel_desc = arcpy.Describe(tmk_parcels_path)
        parcel_fields = [(f.name, f.type, f.length) for f in arcpy.ListFields(tmk_parcels_path)]
        
        print(f"  Records: {parcel_count:,}")
        print(f"  Geometry: {parcel_desc.shapeType}")
        
        # Find TMK-related fields
        tmk_related_fields = [f for f in parcel_fields if 'tmk' in f[0].lower()]
        print(f"  TMK-related fields:")
        for field_info in tmk_related_fields:
            print(f"    {field_info[0]} ({field_info[1]}, length: {field_info[2]})")
        
        # Sample values from the first TMK field
        if tmk_related_fields:
            primary_tmk_field = tmk_related_fields[0][0]
            print(f"  Sample {primary_tmk_field} values:")
            
            with arcpy.da.SearchCursor(tmk_parcels_path, [primary_tmk_field]) as cursor:
                for i, row in enumerate(cursor):
                    if i < 8:
                        print(f"    {row[0]} (type: {type(row[0])})")
                    if i >= 7:
                        break
        
        # Check for Maui parcels specifically
        if tmk_related_fields:
            maui_count = 0
            with arcpy.da.SearchCursor(tmk_parcels_path, [primary_tmk_field]) as cursor:
                for row in cursor:
                    if str(row[0]).startswith('2'):
                        maui_count += 1
                        if maui_count >= 20000:  # Stop counting at reasonable number
                            break
            
            print(f"  Estimated Maui parcels (TMK starting with '2'): {maui_count:,}")
    
    except Exception as e:
        print(f"  Error accessing parcel data: {str(e)}")
        
else:
    print("  TMK parcel polygons NOT FOUND")
    print("  Searched in:")
    for path in possible_paths:
        print(f"    {path}")

# Compare TMK formats between datasets
print(f"\nTMK FORMAT COMPARISON:")
print("=" * 30)

if arcpy.Exists(master_table_path) and tmk_parcels_path and arcpy.Exists(tmk_parcels_path):
    # Get a few TMK values from each dataset
    cesspool_tmks = []
    with arcpy.da.SearchCursor(master_table_path, ['TMK']) as cursor:
        for i, row in enumerate(cursor):
            if i < 10:
                cesspool_tmks.append(str(row[0]))
            if i >= 9:
                break
    
    parcel_tmks = []
    if tmk_related_fields:
        with arcpy.da.SearchCursor(tmk_parcels_path, [primary_tmk_field]) as cursor:
            for i, row in enumerate(cursor):
                if i < 10:
                    parcel_tmks.append(str(row[0]))
                if i >= 9:
                    break
    
    print("Cesspool TMK format:")
    for tmk in cesspool_tmks[:5]:
        print(f"  '{tmk}' (length: {len(tmk)})")
    
    print("Parcel TMK format:")
    for tmk in parcel_tmks[:5]:
        print(f"  '{tmk}' (length: {len(tmk)})")
    
    # Look for potential matches
    matches = set(cesspool_tmks) & set(parcel_tmks)
    print(f"\nDirect TMK matches found: {len(matches)}")
    if matches:
        print("Sample matches:")
        for match in list(matches)[:3]:
            print(f"  {match}")

print(f"\n" + "=" * 70)
print("DATA STRUCTURE ANALYSIS COMPLETE")
print("=" * 70)
print("Status: Bad outputs cleaned up")
print("Next: Develop corrected parcel extraction approach based on TMK format analysis")


CLEANUP BAD OUTPUTS AND EXAMINE DATA STRUCTURES
Cleaning up problematic output files...
  Deleted: Cesspool_Parcel_Polygons.shp
  Deleted: Maui_Cesspool_Polygons.shp
  Deleted: temp_parcels.shp
  Deleted: cesspool_tmks.dbf
  Deleted: Cesspool_Parcels_Statewide.shp
  Deleted: Maui_Cesspool_Parcels.shp
✓ Cleanup complete

EXAMINING DATA STRUCTURE COMPATIBILITY
CESSPOOL POINTS DATA (TMK_Master_Attributes.shp):
  Records: 82,141
  TMK field details:
    Name: TMK
    Type: Integer
    Length: 10
  Sample TMK values:
    186006001 (Oahu)
    186006001 (Oahu)
    186006001 (Oahu)
    186006001 (Oahu)
    186006001 (Oahu)
    186006001 (Oahu)
    186006001 (Oahu)
    186006001 (Oahu)

TMK PARCEL POLYGONS DATA:
  Found at: C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\data\tmk_state.shp
  Error accessing parcel data: ERROR 000229: Cannot open C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\data\tmk_state.shp
Failed to execute (GetCount).


TMK FORMAT COMPARISON:


<class 'NameError'>: name 'tmk_related_fields' is not defined

In [19]:
print("\n" + "=" * 70)
print("FIX DATA ACCESS AND CONTINUE TMK ANALYSIS")
print("=" * 70)

# The cesspool data is good - TMK as Integer, 82,141 records
print("CESSPOOL DATA CONFIRMED:")
print("  TMK Field: Integer type, 10 length")
print("  Sample format: 186006001 (9 digits)")
print("  Records: 82,141")

print(f"\nFINDING TMK PARCEL POLYGONS...")

# Search more thoroughly for the parcel polygons
parcel_candidates = []

# Check in geodatabase first
gdb_path = os.path.join(workspace, "ParcelAnalysis.gdb")
if arcpy.Exists(gdb_path):
    print(f"Checking geodatabase: {gdb_path}")
    arcpy.env.workspace = gdb_path
    feature_classes = arcpy.ListFeatureClasses()
    
    for fc in feature_classes:
        if 'tmk' in fc.lower() or 'parcel' in fc.lower():
            fc_path = os.path.join(gdb_path, fc)
            try:
                desc = arcpy.Describe(fc_path)
                count = int(arcpy.management.GetCount(fc_path)[0])
                parcel_candidates.append({
                    'name': fc,
                    'path': fc_path,
                    'type': desc.shapeType,
                    'count': count
                })
                print(f"  Found: {fc} - {desc.shapeType}, {count:,} records")
            except Exception as e:
                print(f"  Could not access {fc}: {str(e)}")

# Check data directory more thoroughly
print(f"\nSearching data directory recursively...")
data_dir = os.path.join(workspace, "data")

for root, dirs, files in os.walk(data_dir):
    for file in files:
        if (('tmk' in file.lower() or 'parcel' in file.lower()) and 
            file.endswith('.shp')):
            file_path = os.path.join(root, file)
            try:
                desc = arcpy.Describe(file_path)
                if desc.shapeType == 'Polygon':
                    count = int(arcpy.management.GetCount(file_path)[0])
                    parcel_candidates.append({
                        'name': file,
                        'path': file_path,
                        'type': desc.shapeType,
                        'count': count
                    })
                    print(f"  Found: {file} - {desc.shapeType}, {count:,} records")
            except Exception as e:
                print(f"  Could not access {file}: {str(e)}")

# Select the best parcel dataset
if parcel_candidates:
    # Choose the one with most records (likely most comprehensive)
    best_parcel = max(parcel_candidates, key=lambda x: x['count'])
    tmk_parcels_path = best_parcel['path']
    
    print(f"\nUSING PARCEL DATASET:")
    print(f"  Name: {best_parcel['name']}")
    print(f"  Path: {tmk_parcels_path}")
    print(f"  Type: {best_parcel['type']}")
    print(f"  Records: {best_parcel['count']:,}")
    
    # Now examine the TMK field structure
    try:
        parcel_fields = [(f.name, f.type, f.length) for f in arcpy.ListFields(tmk_parcels_path)]
        tmk_related_fields = [f for f in parcel_fields if 'tmk' in f[0].lower()]
        
        print(f"\nTMK FIELDS IN PARCEL DATA:")
        for field_info in tmk_related_fields:
            print(f"  {field_info[0]}: {field_info[1]} (length: {field_info[2]})")
        
        # Sample the TMK values
        if tmk_related_fields:
            primary_tmk_field = tmk_related_fields[0][0]
            print(f"\nSample {primary_tmk_field} values:")
            
            with arcpy.da.SearchCursor(tmk_parcels_path, [primary_tmk_field]) as cursor:
                for i, row in enumerate(cursor):
                    if i < 8:
                        print(f"  {row[0]} (type: {type(row[0])})")
                    if i >= 7:
                        break
            
            print(f"\nTMK FORMAT COMPARISON:")
            print("=" * 30)
            print("Cesspool TMKs: Integer (186006001)")
            print(f"Parcel TMKs: {tmk_related_fields[0][1]} ({primary_tmk_field})")
            
            # Test for direct matches
            cesspool_sample = set()
            with arcpy.da.SearchCursor(master_table_path, ['TMK']) as cursor:
                for i, row in enumerate(cursor):
                    if i < 100:  # Sample first 100
                        cesspool_sample.add(row[0])
                    if i >= 99:
                        break
            
            parcel_sample = set()
            with arcpy.da.SearchCursor(tmk_parcels_path, [primary_tmk_field]) as cursor:
                for i, row in enumerate(cursor):
                    if i < 100:  # Sample first 100
                        parcel_sample.add(row[0])
                    if i >= 99:
                        break
            
            matches = cesspool_sample & parcel_sample
            print(f"Direct matches in sample: {len(matches)}/100")
            
            if matches:
                print("Sample matches found - TMK formats are compatible!")
                print(f"Ready to extract parcel polygons for {len(cesspool_sample)} cesspools")
            else:
                print("No direct matches - need to investigate TMK format differences")
                print("Cesspool sample:", list(cesspool_sample)[:3])
                print("Parcel sample:", list(parcel_sample)[:3])

    except Exception as e:
        print(f"Error examining parcel TMK fields: {str(e)}")

else:
    print("No suitable parcel polygon datasets found")

print(f"\n" + "=" * 70)
print("TMK DATA ANALYSIS COMPLETE")
print("Next step will depend on TMK format compatibility results")
print("=" * 70)


FIX DATA ACCESS AND CONTINUE TMK ANALYSIS
CESSPOOL DATA CONFIRMED:
  TMK Field: Integer type, 10 length
  Sample format: 186006001 (9 digits)
  Records: 82,141

FINDING TMK PARCEL POLYGONS...
Checking geodatabase: C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\ParcelAnalysis.gdb
  Found: Parcels_Hawaii - Polygon, 135,471 records
  Found: tmk_state - Polygon, 384,262 records
  Found: Parcels_Honolulu - Polygon, 171,905 records
  Found: Parcels_Kauai - Polygon, 25,121 records
  Found: Parcels_Maui - Polygon, 51,765 records

Searching data directory recursively...
  Found: tmk_state.shp - Polygon, 384,262 records

USING PARCEL DATASET:
  Name: tmk_state
  Path: C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\ParcelAnalysis.gdb\tmk_state
  Type: Polygon
  Records: 384,262

TMK FIELDS IN PARCEL DATA:
  TMK: Integer (length: 4)
  TMK_txt: String (length: 9)
  cty_tmk: String (length: 8)

Sample TMK values:
  389014053 (type: <class 'int'>)
  389014054 (type: <cl

In [20]:
print("\n" + "=" * 70)
print("DIAGNOSE TMK MISMATCH AND FIND SOLUTION")
print("=" * 70)

# Use the identified best datasets
tmk_parcels_path = r"C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\ParcelAnalysis.gdb\tmk_state"
cesspool_points_path = os.path.join(foundation_dir, "TMK_Master_Attributes.shp")

print("INVESTIGATING TMK MISMATCH...")

# Get larger samples for pattern analysis
print("Collecting larger TMK samples for pattern analysis...")

cesspool_tmks = []
island_tmks = {}

with arcpy.da.SearchCursor(cesspool_points_path, ['TMK', 'Island']) as cursor:
    for row in cursor:
        tmk = row[0]
        island = row[1] if row[1] else 'Unknown'
        cesspool_tmks.append(tmk)
        
        if island not in island_tmks:
            island_tmks[island] = []
        if len(island_tmks[island]) < 5:
            island_tmks[island].append(tmk)

print("Cesspool TMKs by island:")
for island, tmks in island_tmks.items():
    print(f"  {island}: {tmks}")

# Get parcel TMK samples
parcel_tmks = []
with arcpy.da.SearchCursor(tmk_parcels_path, ['TMK']) as cursor:
    for i, row in enumerate(cursor):
        if i < 500:  # Larger sample
            parcel_tmks.append(row[0])
        if i >= 499:
            break

print(f"\nParcel TMK sample (first 10): {parcel_tmks[:10]}")

# Analyze TMK patterns
print(f"\nTMK PATTERN ANALYSIS:")
print("=" * 30)

# Check if cesspool TMKs are a subset of parcel TMKs
cesspool_set = set(cesspool_tmks[:1000])  # Sample first 1000
parcel_set = set(parcel_tmks)

matches = cesspool_set & parcel_set
print(f"Direct matches found: {len(matches)} out of {len(cesspool_set)} cesspool TMKs")

if matches:
    print("SOLUTION: Direct TMK matching will work")
    print(f"Sample matches: {list(matches)[:5]}")
else:
    print("NO DIRECT MATCHES - investigating alternative approaches...")
    
    # Check if there's a systematic pattern difference
    print("\nChecking for systematic TMK differences...")
    
    # Look at TMK ranges by island
    cesspool_ranges = {}
    for island, tmks in island_tmks.items():
        if tmks:
            cesspool_ranges[island] = {
                'min': min(tmks),
                'max': max(tmks),
                'first_digit': str(tmks[0])[0]
            }
    
    print("Cesspool TMK ranges by island:")
    for island, info in cesspool_ranges.items():
        print(f"  {island}: {info['min']} - {info['max']} (starts with {info['first_digit']})")
    
    # Check parcel TMK ranges  
    parcel_first_digits = {}
    for tmk in parcel_tmks[:100]:
        first_digit = str(tmk)[0]
        if first_digit not in parcel_first_digits:
            parcel_first_digits[first_digit] = []
        if len(parcel_first_digits[first_digit]) < 3:
            parcel_first_digits[first_digit].append(tmk)
    
    print("Parcel TMK patterns by first digit:")
    for digit, examples in parcel_first_digits.items():
        print(f"  {digit}xxx: {examples}")

# Alternative approach: Spatial relationship
print(f"\nALTERNATIVE: SPATIAL RELATIONSHIP APPROACH")
print("=" * 50)
print("Since TMK direct matching may not work, use spatial containment:")
print("1. Use 'Select Layer by Location' with CONTAINS relationship")
print("2. Select parcels that contain cesspool points (centroids)")
print("3. This should work regardless of TMK numbering differences")

# Test spatial approach on small sample
print("\nTesting spatial approach with small sample...")

try:
    # Create a small sample for testing
    test_sample_path = os.path.join(foundation_dir, "Test_Cesspool_Sample.shp")
    
    # Select first 100 cesspool points for testing
    arcpy.management.MakeFeatureLayer(cesspool_points_path, "cesspool_layer")
    arcpy.management.SelectLayerByAttribute("cesspool_layer", "NEW_SELECTION", "FID < 100")
    arcpy.management.CopyFeatures("cesspool_layer", test_sample_path)
    
    test_count = int(arcpy.management.GetCount(test_sample_path)[0])
    print(f"Created test sample: {test_count} cesspool points")
    
    # Test spatial selection
    arcpy.management.MakeFeatureLayer(tmk_parcels_path, "parcel_layer")
    
    # Select parcels that contain the test cesspool points
    arcpy.management.SelectLayerByLocation(
        in_layer="parcel_layer",
        overlap_type="CONTAINS",
        select_features=test_sample_path,
        selection_type="NEW_SELECTION"
    )
    
    selected_count = int(arcpy.management.GetCount("parcel_layer")[0])
    print(f"Spatial selection result: {selected_count} parcels contain {test_count} cesspool points")
    
    if selected_count > 0:
        print("✓ SPATIAL APPROACH WORKS!")
        print("Recommendation: Use spatial relationship instead of TMK matching")
    else:
        print("⚠ Spatial approach also failed - need to investigate further")
    
    # Clean up test files
    arcpy.management.Delete("cesspool_layer")
    arcpy.management.Delete("parcel_layer") 
    arcpy.management.Delete(test_sample_path)
    
except Exception as e:
    print(f"Error testing spatial approach: {str(e)}")

print(f"\n" + "=" * 70)
print("TMK DIAGNOSIS COMPLETE")
print("=" * 70)
print("Next: Implement working parcel extraction method")


DIAGNOSE TMK MISMATCH AND FIND SOLUTION
INVESTIGATING TMK MISMATCH...
Cesspool TMKs by island:
  Oahu: [186006001, 186006001, 186006001, 186006001, 186006001]
  Kauai: [412010022, 412010039, 412010053, 412010110, 412010138]
  Maui: [221019029, 221019033, 221019034, 221019035, 221019036]
  BI: [311002003, 311002005, 311002010, 311002011, 311002012]

Parcel TMK sample (first 10): [389014053, 389014054, 389014055, 389014056, 388009112, 388009113, 388009114, 388009115, 388009116, 388009117]

TMK PATTERN ANALYSIS:
Direct matches found: 0 out of 970 cesspool TMKs
NO DIRECT MATCHES - investigating alternative approaches...

Checking for systematic TMK differences...
Cesspool TMK ranges by island:
  Oahu: 186006001 - 186006001 (starts with 1)
  Kauai: 412010022 - 412010138 (starts with 4)
  Maui: 221019029 - 221019036 (starts with 2)
  BI: 311002003 - 311002012 (starts with 3)
Parcel TMK patterns by first digit:
  3xxx: [389014053, 389014054, 389014055]

ALTERNATIVE: SPATIAL RELATIONSHIP APPR

In [21]:
print("\n" + "=" * 70)
print("EXTRACT CESSPOOL PARCEL POLYGONS - SPATIAL APPROACH")
print("=" * 70)

# Use spatial relationship to get parcel polygons for cesspools
tmk_parcels_path = r"C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\ParcelAnalysis.gdb\tmk_state"
cesspool_points_path = os.path.join(foundation_dir, "TMK_Master_Attributes.shp")
cesspool_parcels_path = os.path.join(foundation_dir, "Cesspool_Parcel_Polygons_Spatial.shp")

print("Using spatial relationship approach...")
print(f"Parcel polygons: {tmk_parcels_path} ({384262:,} parcels)")
print(f"Cesspool points: {cesspool_points_path} ({82141:,} points)")

try:
    # Create feature layers for spatial operations
    arcpy.management.MakeFeatureLayer(tmk_parcels_path, "parcel_layer")
    arcpy.management.MakeFeatureLayer(cesspool_points_path, "cesspool_layer")
    
    print("Selecting parcels that contain cesspool points...")
    print("This may take a few minutes for 82,000+ cesspools...")
    
    # Select parcels that contain cesspool points
    arcpy.management.SelectLayerByLocation(
        in_layer="parcel_layer",
        overlap_type="CONTAINS",
        select_features="cesspool_layer",
        selection_type="NEW_SELECTION"
    )
    
    # Get count of selected parcels
    selected_count = int(arcpy.management.GetCount("parcel_layer")[0])
    print(f"Selected {selected_count:,} parcel polygons containing cesspools")
    
    if selected_count > 0:
        # Copy selected parcels to new dataset
        print("Creating cesspool parcel polygon dataset...")
        arcpy.management.CopyFeatures("parcel_layer", cesspool_parcels_path)
        
        # Verify the result
        result_count = int(arcpy.management.GetCount(cesspool_parcels_path)[0])
        result_desc = arcpy.Describe(cesspool_parcels_path)
        
        print(f"✓ Created: {result_count:,} cesspool parcel polygons")
        print(f"✓ Type: {result_desc.shapeType}")
        print(f"✓ Location: {cesspool_parcels_path}")
        
        # Check the parcel TMK fields we now have
        parcel_fields = [f.name for f in arcpy.ListFields(cesspool_parcels_path)]
        tmk_fields = [f for f in parcel_fields if 'tmk' in f.lower()]
        print(f"Available TMK fields in parcels: {tmk_fields}")
        
        # Sample the results
        print("\nSample cesspool parcel polygons:")
        sample_fields = ['TMK', 'Shape_Area'] if 'TMK' in parcel_fields else [tmk_fields[0], 'Shape_Area']
        
        with arcpy.da.SearchCursor(cesspool_parcels_path, sample_fields) as cursor:
            for i, row in enumerate(cursor):
                if i < 5:
                    area_acres = row[1] / 43560 if row[1] else 0  # Convert sq ft to acres
                    print(f"  Parcel TMK: {row[0]}, Area: {area_acres:.2f} acres")
                if i >= 4:
                    break
        
        # Now we need to associate these parcel polygons back to cesspool data
        print(f"\nAssociating parcel polygons with cesspool TMK data...")
        
        # Spatial join to add cesspool attributes to parcel polygons
        cesspool_parcels_with_data_path = os.path.join(foundation_dir, "Cesspool_Parcels_With_Attributes.shp")
        
        arcpy.analysis.SpatialJoin(
            target_features=cesspool_parcels_path,        # Parcel polygons
            join_features=cesspool_points_path,           # Cesspool points with TMK data
            out_feature_class=cesspool_parcels_with_data_path,
            join_operation="JOIN_ONE_TO_ONE",
            join_type="KEEP_ALL",
            match_option="CONTAINS"                       # Parcel contains cesspool point
        )
        
        final_count = int(arcpy.management.GetCount(cesspool_parcels_with_data_path)[0])
        print(f"✓ Created final dataset: {final_count:,} parcels with cesspool attributes")
        
        # Verify the joined data
        final_fields = [f.name for f in arcpy.ListFields(cesspool_parcels_with_data_path)]
        cesspool_fields = ['TMK', 'Island', 'dist2_MunW', 'dist2_DomW']
        preserved_fields = [f for f in cesspool_fields if f in final_fields]
        
        print(f"Preserved cesspool attributes: {preserved_fields}")
        print(f"Total fields in final dataset: {len(final_fields)}")
        
        # Sample the final integrated data
        print("\nSample integrated cesspool-parcel data:")
        display_fields = ['TMK', 'Island', 'dist2_MunW', 'Shape_Area']
        display_fields = [f for f in display_fields if f in final_fields]
        
        with arcpy.da.SearchCursor(cesspool_parcels_with_data_path, display_fields) as cursor:
            for i, row in enumerate(cursor):
                if i < 3:
                    display_data = []
                    for j, value in enumerate(row):
                        field_name = display_fields[j]
                        if 'area' in field_name.lower() and value:
                            display_value = f"{value/43560:.2f} acres"
                        elif 'dist' in field_name.lower() and value:
                            display_value = f"{value:.0f}m"
                        else:
                            display_value = str(value)
                        display_data.append(f"{field_name}: {display_value}")
                    print(f"  {' | '.join(display_data)}")
                if i >= 2:
                    break
        
        # Create Maui subset for pilot
        print(f"\nCreating Maui subset for area-weighted pilot...")
        maui_cesspool_parcels_path = os.path.join(foundation_dir, "Maui_Cesspool_Parcels_Final.shp")
        
        # Use cesspool TMK for Maui selection (starts with 2)
        maui_expression = "TMK LIKE '2%'" if 'TMK' in final_fields else f"{preserved_fields[0]} LIKE '2%'"
        
        arcpy.analysis.Select(cesspool_parcels_with_data_path, maui_cesspool_parcels_path, maui_expression)
        
        maui_count = int(arcpy.management.GetCount(maui_cesspool_parcels_path)[0])
        print(f"✓ Maui pilot dataset: {maui_count:,} cesspool parcels")
        
        print(f"\n" + "=" * 70)
        print("CESSPOOL PARCEL EXTRACTION SUCCESS!")
        print("=" * 70)
        print("Created datasets:")
        print(f"  Statewide: {cesspool_parcels_with_data_path}")
        print(f"  Maui pilot: {maui_cesspool_parcels_path}")
        print("Each parcel polygon now has:")
        print("  - Parcel geometry for area calculations")
        print("  - Cesspool TMK and attribute data")  
        print("  - Wells distance information")
        print("Ready for area-weighted spatial analysis!")
        
    else:
        print("ERROR: No parcels selected - spatial relationship failed")
        
    # Clean up layers
    arcpy.management.Delete("parcel_layer")
    arcpy.management.Delete("cesspool_layer")
    
except Exception as e:
    print(f"ERROR in spatial parcel extraction: {str(e)}")


EXTRACT CESSPOOL PARCEL POLYGONS - SPATIAL APPROACH
Using spatial relationship approach...
Parcel polygons: C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\ParcelAnalysis.gdb\tmk_state (384,262 parcels)
Cesspool points: C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\outputs\foundation\TMK_Master_Attributes.shp (82,141 points)
Selecting parcels that contain cesspool points...
This may take a few minutes for 82,000+ cesspools...
Selected 68,565 parcel polygons containing cesspools
Creating cesspool parcel polygon dataset...
✓ Created: 68,565 cesspool parcel polygons
✓ Type: Polygon
✓ Location: C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\outputs\foundation\Cesspool_Parcel_Polygons_Spatial.shp
Available TMK fields in parcels: ['TMK', 'TMK_txt', 'cty_tmk']

Sample cesspool parcel polygons:
  Parcel TMK: 389014053, Area: 0.02 acres
  Parcel TMK: 389014054, Area: 0.02 acres
  Parcel TMK: 389014055, Area: 0.02 acres
  Parcel TMK: 389014056, Area:

In [23]:
print("\n" + "=" * 50)
print("CREATE MAUI SUBSET - FIXED SQL EXPRESSION")
print("=" * 50)

# Define paths
foundation_dir = os.path.join(workspace, "outputs", "foundation")
cesspool_parcels_path = os.path.join(foundation_dir, "Cesspool_Parcels_With_Attributes.shp")
maui_subset_path = os.path.join(foundation_dir, "Maui_Cesspool_Parcels_Pilot.shp")

print(f"Source dataset: {cesspool_parcels_path}")

if arcpy.Exists(cesspool_parcels_path):
    # Check the TMK field type first
    fields = arcpy.ListFields(cesspool_parcels_path)
    tmk_field_info = None
    
    for field in fields:
        if field.name == 'TMK':
            tmk_field_info = field
            break
    
    if tmk_field_info:
        print(f"TMK field type: {tmk_field_info.type}")
        
        # Since TMK is Integer type, we need to use numeric comparison
        # Maui TMKs start with 2, so they're in range 200000000-299999999
        if tmk_field_info.type in ['Integer', 'OID']:
            maui_expression = "TMK >= 200000000 AND TMK < 300000000"
            print(f"Using numeric expression for Integer TMK: {maui_expression}")
        else:
            maui_expression = "TMK LIKE '2%'"
            print(f"Using text expression: {maui_expression}")
        
        print("Creating Maui subset...")
        
        try:
            arcpy.analysis.Select(cesspool_parcels_path, maui_subset_path, maui_expression)
            
            # Verify results
            maui_count = int(arcpy.management.GetCount(maui_subset_path)[0])
            print(f"Success! Maui subset created: {maui_count:,} parcels")
            
            # Sample the results
            print("Sample Maui cesspool parcels:")
            sample_fields = ['TMK', 'Island'] if 'Island' in [f.name for f in fields] else ['TMK']
            
            with arcpy.da.SearchCursor(maui_subset_path, sample_fields) as cursor:
                for i, row in enumerate(cursor):
                    if i < 5:
                        if len(sample_fields) > 1:
                            print(f"  TMK: {row[0]}, Island: {row[1]}")
                        else:
                            print(f"  TMK: {row[0]}")
                    if i >= 4:
                        break
            
            print(f"\nMaui pilot dataset ready: {maui_subset_path}")
            print(f"Ready for area-weighted spatial analysis!")
            
        except Exception as e:
            print(f"Error creating Maui subset: {str(e)}")
    
    else:
        print("TMK field not found in dataset")

else:
    print(f"Source dataset not found: {cesspool_parcels_path}")
    print("Need to run the spatial parcel extraction first")


CREATE MAUI SUBSET - FIXED SQL EXPRESSION
Source dataset: C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\outputs\foundation\Cesspool_Parcels_With_Attributes.shp
TMK field type: Integer
Using numeric expression for Integer TMK: TMK >= 200000000 AND TMK < 300000000
Creating Maui subset...
Success! Maui subset created: 8,248 parcels
Sample Maui cesspool parcels:
  TMK: 211003003
  TMK: 211003012
  TMK: 211003030
  TMK: 211003031
  TMK: 211003041

Maui pilot dataset ready: C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\outputs\foundation\Maui_Cesspool_Parcels_Pilot.shp
Ready for area-weighted spatial analysis!


In [24]:
print("\n" + "=" * 50)
print("EXAMINE BEDROOMS_OUT.CSV COVERAGE")
print("=" * 50)

import pandas as pd

# Read the bedrooms file
bedrooms_file = os.path.join(data_dir, "bedrooms_out.csv")

if os.path.exists(bedrooms_file):
    print(f"Reading: {bedrooms_file}")
    
    try:
        df_bedrooms = pd.read_csv(bedrooms_file)
        print(f"Total records: {len(df_bedrooms):,}")
        print(f"Columns: {list(df_bedrooms.columns)}")
        
        # Check TMK format and coverage
        print(f"\nTMK analysis:")
        print(f"Sample TMK values: {df_bedrooms['TMK'].head().tolist()}")
        print(f"TMK data type: {df_bedrooms['TMK'].dtype}")
        
        # Determine geographic coverage by TMK first digit
        df_bedrooms['TMK_str'] = df_bedrooms['TMK'].astype(str)
        df_bedrooms['first_digit'] = df_bedrooms['TMK_str'].str[0]
        
        coverage = df_bedrooms['first_digit'].value_counts().sort_index()
        print(f"\nGeographic coverage by TMK first digit:")
        
        island_mapping = {
            '1': 'Big Island (Hawaii County)', 
            '2': 'Maui County',
            '3': 'Honolulu County (Oahu)',
            '4': 'Kauai County'
        }
        
        for digit, count in coverage.items():
            island = island_mapping.get(digit, f'Unknown ({digit})')
            print(f"  {digit}xxx: {count:,} records ({island})")
        
        # Check bedroom count distribution
        print(f"\nBedroom count distribution:")
        bedroom_dist = df_bedrooms['BED_ROOMS'].value_counts().sort_index()
        for bedrooms, count in bedroom_dist.head(10).items():
            print(f"  {bedrooms} bedrooms: {count:,} parcels")
        
        if len(bedroom_dist) > 10:
            print(f"  ... and {len(bedroom_dist) - 10} other bedroom counts")
        
        # Check for missing/zero bedrooms
        zero_bedrooms = (df_bedrooms['BED_ROOMS'] == 0).sum()
        null_bedrooms = df_bedrooms['BED_ROOMS'].isna().sum()
        print(f"\nData quality:")
        print(f"  Zero bedrooms: {zero_bedrooms:,} parcels")
        print(f"  Null/missing: {null_bedrooms:,} parcels")
        
        # Sample records by island
        print(f"\nSample records by island:")
        for digit in ['1', '2', '3', '4']:
            island_data = df_bedrooms[df_bedrooms['first_digit'] == digit].head(3)
            if len(island_data) > 0:
                island_name = island_mapping[digit]
                print(f"\n{island_name}:")
                for _, row in island_data.iterrows():
                    print(f"  TMK: {row['TMK']}, Bedrooms: {row['BED_ROOMS']}")
        
        # Determine coverage scope
        if len(coverage) >= 4:
            print(f"\n✓ STATEWIDE COVERAGE: All 4 counties represented")
        elif '2' in coverage:
            if len(coverage) == 1:
                print(f"\n⚠ MAUI ONLY: Only Maui County data present")
            else:
                print(f"\n⚠ PARTIAL COVERAGE: {len(coverage)} counties present")
        else:
            print(f"\n⚠ LIMITED COVERAGE: Maui not included")

    except Exception as e:
        print(f"Error reading bedrooms file: {str(e)}")

else:
    print(f"Bedrooms file not found: {bedrooms_file}")

print(f"\nREADY TO JOIN BEDROOM DATA")
print("If coverage matches our cesspool data, we can proceed with TMK join")



EXAMINE BEDROOMS_OUT.CSV COVERAGE
Reading: C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\data\bedrooms_out.csv
Total records: 70,322
Columns: ['TMK', 'BED_ROOMS']

TMK analysis:
Sample TMK values: [2110010230000, 2110020090000, 2110030030000, 2110030060000, 2110030120000]
TMK data type: int64

Geographic coverage by TMK first digit:
  2xxx: 70,322 records (Maui County)

Bedroom count distribution:
  0 bedrooms: 104 parcels
  1 bedrooms: 13,336 parcels
  2 bedrooms: 20,277 parcels
  3 bedrooms: 24,339 parcels
  4 bedrooms: 7,831 parcels
  5 bedrooms: 2,661 parcels
  6 bedrooms: 951 parcels
  7 bedrooms: 362 parcels
  8 bedrooms: 222 parcels
  9 bedrooms: 115 parcels
  ... and 8 other bedroom counts

Data quality:
  Zero bedrooms: 104 parcels
  Null/missing: 0 parcels

Sample records by island:

Maui County:
  TMK: 2110010230000, Bedrooms: 1
  TMK: 2110020090000, Bedrooms: 3
  TMK: 2110030030000, Bedrooms: 3

⚠ MAUI ONLY: Only Maui County data present

READY TO JOIN BEDR

In [25]:
import arcpy
import os

# Set your geodatabase path
gdb_path = r"C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\ParcelAnalysis.gdb"

# Feature service URL
service_url = "https://services1.arcgis.com/x4h61KaW16vFs7PM/arcgis/rest/services/Categories_HI_Bedrooms_Per_Ac/FeatureServer/0"

# Output name in your geodatabase
output_name = "HI_Parcels_Bedrooms_Bathrooms_Units"

print("Starting download from ArcGIS Online...")
print(f"Source: {service_url}")
print(f"Destination: {os.path.join(gdb_path, output_name)}")

try:
    # Copy features from service to local geodatabase
    arcpy.management.CopyFeatures(
        service_url, 
        os.path.join(gdb_path, output_name)
    )
    
    # Get record count
    result = arcpy.management.GetCount(os.path.join(gdb_path, output_name))
    record_count = int(result[0])
    
    print(f"✅ SUCCESS: Downloaded {record_count:,} records")
    print(f"✅ Saved to: {output_name}")
    print("✅ Ready for Matrix analysis!")
    
except Exception as e:
    print(f"❌ Error: {str(e)}")
    print("Try Method 2 below...")

Starting download from ArcGIS Online...
Source: https://services1.arcgis.com/x4h61KaW16vFs7PM/arcgis/rest/services/Categories_HI_Bedrooms_Per_Ac/FeatureServer/0
Destination: C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\ParcelAnalysis.gdb\HI_Parcels_Bedrooms_Bathrooms_Units
✅ SUCCESS: Downloaded 384,209 records
✅ Saved to: HI_Parcels_Bedrooms_Bathrooms_Units
✅ Ready for Matrix analysis!


In [26]:
import arcpy
import os

print("=== TMK COMPATIBILITY CHECK ===")
print("Comparing cesspool data vs bedroom data for TMK matching")
print()

# Set paths to your two key datasets
gdb_path = r"C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\ParcelAnalysis.gdb"
cesspool_fc = os.path.join(gdb_path, "Cesspool_Parcels_With_Attributes")
bedroom_fc = os.path.join(gdb_path, "HI_Parcels_Bedrooms_Bathrooms_Units")

# Check if both datasets exist
cesspool_exists = arcpy.Exists(cesspool_fc)
bedroom_exists = arcpy.Exists(bedroom_fc)

print(f"Cesspool data exists: {cesspool_exists}")
print(f"Bedroom data exists: {bedroom_exists}")
print()

if cesspool_exists and bedroom_exists:
    # Get record counts
    cesspool_count = int(arcpy.management.GetCount(cesspool_fc)[0])
    bedroom_count = int(arcpy.management.GetCount(bedroom_fc)[0])
    
    print(f"Cesspool parcels: {cesspool_count:,}")
    print(f"Bedroom parcels: {bedroom_count:,}")
    print()
    
    # Sample TMK formats from both datasets
    print("--- CESSPOOL DATA TMK SAMPLES ---")
    with arcpy.da.SearchCursor(cesspool_fc, ["TMK"]) as cursor:
        cesspool_tmks = []
        for i, row in enumerate(cursor):
            if i < 5:
                cesspool_tmks.append(str(row[0]))
            else:
                break
    
    for i, tmk in enumerate(cesspool_tmks):
        print(f"  {i+1}. TMK: {tmk} (Length: {len(tmk)})")
    
    print("\n--- BEDROOM DATA TMK SAMPLES ---")
    with arcpy.da.SearchCursor(bedroom_fc, ["TMK", "SUM_Bedrooms"]) as cursor:
        bedroom_tmks = []
        bedroom_counts = []
        for i, row in enumerate(cursor):
            if i < 5:
                bedroom_tmks.append(str(row[0]))
                bedroom_counts.append(row[1])
            else:
                break
    
    for i, (tmk, bedrooms) in enumerate(zip(bedroom_tmks, bedroom_counts)):
        print(f"  {i+1}. TMK: {tmk} (Length: {len(tmk)}) | Bedrooms: {bedrooms}")
    
    # Check TMK format compatibility
    print("\n--- TMK FORMAT COMPATIBILITY ---")
    cesspool_lengths = [len(tmk) for tmk in cesspool_tmks]
    bedroom_lengths = [len(tmk) for tmk in bedroom_tmks]
    
    print(f"Cesspool TMK lengths: {set(cesspool_lengths)}")
    print(f"Bedroom TMK lengths: {set(bedroom_lengths)}")
    
    if set(cesspool_lengths) == set(bedroom_lengths):
        print("✅ TMK formats MATCH - Direct join possible!")
    else:
        print("⚠️  TMK formats DIFFER - Conversion needed")
    
    # Test for actual TMK overlap
    print("\n--- TMK OVERLAP TEST ---")
    cesspool_tmk_set = set(cesspool_tmks)
    bedroom_tmk_set = set(bedroom_tmks)
    overlap = cesspool_tmk_set.intersection(bedroom_tmk_set)
    
    if overlap:
        print(f"✅ Found {len(overlap)} matching TMKs in samples")
        print(f"Matching TMKs: {list(overlap)}")
    else:
        print("⚠️  No matching TMKs found in samples - need broader check")

else:
    print("❌ One or both datasets not found - check import status")

print("\n=== NEXT STEPS ===")
print("If TMK formats match: Ready for spatial join")
print("If TMK formats differ: Need conversion script") 
print("If no overlaps: Check county coverage alignment")

=== TMK COMPATIBILITY CHECK ===
Comparing cesspool data vs bedroom data for TMK matching

Cesspool data exists: False
Bedroom data exists: True

❌ One or both datasets not found - check import status

=== NEXT STEPS ===
If TMK formats match: Ready for spatial join
If TMK formats differ: Need conversion script
If no overlaps: Check county coverage alignment


In [27]:
import requests
import json
import arcpy
import os

print("=== DIRECT API DOWNLOAD ===")

# REST API query URL
base_url = "https://services1.arcgis.com/x4h61KaW16vFs7PM/arcgis/rest/services/Categories_HI_Bedrooms_Per_Ac/FeatureServer/0/query"

# Parameters for getting all data
params = {
    'where': '1=1',  # Get all records
    'outFields': '*',  # All fields
    'f': 'json',      # JSON format
    'returnGeometry': 'true'
}

try:
    print("Querying REST API...")
    response = requests.get(base_url, params=params, timeout=300)
    
    if response.status_code == 200:
        data = response.json()
        print(f"✅ API Response received")
        print(f"Features returned: {len(data.get('features', []))}")
        
        # Show sample feature
        if data.get('features'):
            sample = data['features'][0]['attributes']
            print(f"Sample fields: {list(sample.keys())}")
            print(f"Sample TMK: {sample.get('TMK', 'N/A')}")
            print(f"Sample County: {sample.get('county', 'N/A')}")
            print(f"Sample Bedrooms: {sample.get('SUM_Bedrooms', 'N/A')}")
        
    else:
        print(f"❌ API Error: {response.status_code}")
        
except Exception as e:
    print(f"❌ Request failed: {str(e)}")

=== DIRECT API DOWNLOAD ===
Querying REST API...
✅ API Response received
Features returned: 2000
Sample fields: ['OBJECTID', 'TMK', 'TMK_txt', 'county', 'island', 'cty_tmk', 'GISAcres', 'qpub_link', 'SUM_Bedrooms', 'SUM_Bathrooms', 'COUNT_Units', 'BR_per_AC', 'Cat_BR_per_AC', 'Shape__Area', 'Shape__Length']
Sample TMK: 111001001
Sample County: Honolulu
Sample Bedrooms: 0


In [28]:
import requests
import json
import arcpy
import os

print("=== DOWNLOADING ALL BEDROOM DATA ===")

base_url = "https://services1.arcgis.com/x4h61KaW16vFs7PM/arcgis/rest/services/Categories_HI_Bedrooms_Per_Ac/FeatureServer/0/query"

# First, get the total count
count_params = {
    'where': '1=1',
    'returnCountOnly': 'true',
    'f': 'json'
}

response = requests.get(base_url, params=count_params)
total_count = response.json().get('count', 0)
print(f"Total records available: {total_count:,}")

# Now download in chunks
all_features = []
offset = 0
chunk_size = 2000

while offset < total_count:
    print(f"Downloading records {offset + 1} to {min(offset + chunk_size, total_count)}...")
    
    params = {
        'where': '1=1',
        'outFields': '*',
        'f': 'json',
        'returnGeometry': 'true',
        'resultOffset': offset,
        'resultRecordCount': chunk_size
    }
    
    try:
        response = requests.get(base_url, params=params, timeout=300)
        if response.status_code == 200:
            data = response.json()
            features = data.get('features', [])
            all_features.extend(features)
            print(f"  ✅ Got {len(features)} features")
        else:
            print(f"  ❌ Error: {response.status_code}")
            break
            
    except Exception as e:
        print(f"  ❌ Request failed: {str(e)}")
        break
    
    offset += chunk_size

print(f"\n✅ Total features downloaded: {len(all_features):,}")

# Analyze the complete dataset
if all_features:
    counties = set()
    bedroom_counts = {}
    tmk_samples = []
    
    for feature in all_features[:10]:  # Sample first 10
        attrs = feature['attributes']
        counties.add(attrs.get('county'))
        tmk_samples.append(attrs.get('TMK'))
        
        bedrooms = attrs.get('SUM_Bedrooms', 0)
        if bedrooms > 0:
            bedroom_counts[bedrooms] = bedroom_counts.get(bedrooms, 0) + 1
    
    print(f"\nCounties found: {sorted(counties)}")
    print(f"Sample TMKs: {tmk_samples}")
    print("Bedroom distribution (sample):", dict(list(bedroom_counts.items())[:10]))

=== DOWNLOADING ALL BEDROOM DATA ===
Total records available: 384,209
  ✅ Got 2000 features
  ✅ Got 2000 features
  ✅ Got 2000 features
  ✅ Got 2000 features
  ✅ Got 2000 features
  ✅ Got 2000 features
  ✅ Got 2000 features
  ✅ Got 2000 features
  ✅ Got 2000 features
  ✅ Got 2000 features
  ✅ Got 2000 features
  ✅ Got 2000 features
  ✅ Got 2000 features
  ✅ Got 2000 features
  ✅ Got 2000 features
  ✅ Got 2000 features
  ✅ Got 2000 features
  ✅ Got 2000 features
  ✅ Got 2000 features
  ✅ Got 2000 features
  ✅ Got 2000 features
  ✅ Got 2000 features
  ✅ Got 2000 features
  ✅ Got 2000 features
  ✅ Got 2000 features
  ✅ Got 2000 features
  ✅ Got 2000 features
  ✅ Got 2000 features
  ✅ Got 2000 features
  ✅ Got 2000 features
  ✅ Got 2000 features
  ✅ Got 2000 features
  ✅ Got 2000 features
  ✅ Got 2000 features
  ✅ Got 2000 features
  ✅ Got 2000 features
  ✅ Got 2000 features
  ✅ Got 2000 features
  ✅ Got 2000 features
  ✅ Got 2000 features
  ✅ Got 2000 features
  ✅ Got 2000 features
  ✅ Go

In [29]:
print("\n=== CREATING GEODATABASE FEATURE CLASS ===")

gdb_path = r"C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\ParcelAnalysis.gdb"
fc_name = "HI_Parcels_Bedrooms_Bathrooms_Units"
fc_path = os.path.join(gdb_path, fc_name)

# Create the feature class with proper schema
try:
    # Delete if exists
    if arcpy.Exists(fc_path):
        arcpy.management.Delete(fc_path)
        print("Deleted existing feature class")
    
    # Create new feature class
    arcpy.management.CreateFeatureclass(
        gdb_path, 
        fc_name, 
        "POLYGON", 
        spatial_reference=arcpy.SpatialReference(4326)  # WGS84
    )
    
    # Add fields based on the API response
    fields_to_add = [
        ("TMK", "TEXT", "", "", 20),
        ("county", "TEXT", "", "", 20),
        ("island", "TEXT", "", "", 10),
        ("GISAcres", "DOUBLE", "", "", ""),
        ("SUM_Bedrooms", "LONG", "", "", ""),
        ("SUM_Bathrooms", "LONG", "", "", ""),
        ("COUNT_Units", "LONG", "", "", ""),
        ("BR_per_AC", "DOUBLE", "", "", "")
    ]
    
    for field_name, field_type, field_precision, field_scale, field_length in fields_to_add:
        arcpy.management.AddField(fc_path, field_name, field_type, field_precision, field_scale, field_length)
    
    print(f"✅ Created feature class: {fc_name}")
    print("✅ Added all required fields")
    
    # Now populate with the downloaded data
    print("\n=== POPULATING FEATURE CLASS ===")
    print("This may take several minutes for 384k records...")
    
    with arcpy.da.InsertCursor(fc_path, 
        ["SHAPE@", "TMK", "county", "island", "GISAcres", "SUM_Bedrooms", "SUM_Bathrooms", "COUNT_Units", "BR_per_AC"]) as cursor:
        
        count = 0
        for feature in all_features:
            try:
                attrs = feature['attributes']
                geom = feature['geometry']
                
                # Create polygon from geometry
                if geom and geom.get('rings'):
                    polygon = arcpy.Polygon(
                        arcpy.Array([arcpy.Point(x, y) for x, y in geom['rings'][0]]),
                        arcpy.SpatialReference(4326)
                    )
                    
                    # Insert row
                    cursor.insertRow([
                        polygon,
                        str(attrs.get('TMK', '')),
                        attrs.get('county', ''),
                        attrs.get('island', ''),
                        attrs.get('GISAcres', 0),
                        attrs.get('SUM_Bedrooms', 0),
                        attrs.get('SUM_Bathrooms', 0),
                        attrs.get('COUNT_Units', 0),
                        attrs.get('BR_per_AC', 0)
                    ])
                    
                    count += 1
                    if count % 10000 == 0:
                        print(f"  Inserted {count:,} records...")
                        
            except Exception as e:
                print(f"  Error inserting record {count}: {str(e)}")
                continue
    
    print(f"✅ Successfully populated {count:,} records")
    
    # Verify the result
    final_count = int(arcpy.management.GetCount(fc_path)[0])
    print(f"✅ Final record count in geodatabase: {final_count:,}")
    
except Exception as e:
    print(f"❌ Error: {str(e)}")


=== CREATING GEODATABASE FEATURE CLASS ===
Deleted existing feature class
✅ Created feature class: HI_Parcels_Bedrooms_Bathrooms_Units
✅ Added all required fields

=== POPULATING FEATURE CLASS ===
This may take several minutes for 384k records...
  Inserted 10,000 records...
  Inserted 20,000 records...
  Inserted 30,000 records...
  Inserted 40,000 records...
  Inserted 50,000 records...
  Inserted 60,000 records...
  Inserted 70,000 records...
  Inserted 80,000 records...
  Inserted 90,000 records...
  Inserted 100,000 records...
  Inserted 110,000 records...
  Inserted 120,000 records...
  Inserted 130,000 records...
  Inserted 140,000 records...
  Inserted 150,000 records...
  Inserted 160,000 records...
  Inserted 170,000 records...
  Inserted 180,000 records...
  Inserted 190,000 records...
  Error inserting record 191439: 'geometry'
  Inserted 200,000 records...
  Inserted 210,000 records...
  Inserted 220,000 records...
  Inserted 230,000 records...
  Inserted 240,000 records.

In [30]:
import os
import arcpy

print("=== CHECKING FOUNDATION FOLDER ===")

foundation_folder = r"C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\Outputs\foundation"

print(f"Foundation folder: {foundation_folder}")
print(f"Folder exists: {os.path.exists(foundation_folder)}")

if os.path.exists(foundation_folder):
    print("\nContents of foundation folder:")
    
    files = os.listdir(foundation_folder)
    for i, file in enumerate(files, 1):
        file_path = os.path.join(foundation_folder, file)
        if os.path.isfile(file_path):
            size_mb = os.path.getsize(file_path) / (1024*1024)
            print(f"{i:2}. {file} ({size_mb:.1f} MB)")
    
    # Look for shapefiles or feature classes
    shapefiles = [f for f in files if f.endswith('.shp')]
    if shapefiles:
        print(f"\n--- SHAPEFILES FOUND ---")
        for shp in shapefiles:
            shp_path = os.path.join(foundation_folder, shp)
            try:
                count = int(arcpy.management.GetCount(shp_path)[0])
                print(f"✓ {shp} ({count:,} records)")
                
                # Sample the data
                print(f"  Checking TMK format in {shp}...")
                with arcpy.da.SearchCursor(shp_path, ["*"]) as cursor:
                    fields = cursor.fields
                    print(f"  Fields: {', '.join(fields[:10])}{'...' if len(fields) > 10 else ''}")
                    
                    # Look for TMK field and sample data
                    tmk_fields = [f for f in fields if 'tmk' in f.lower()]
                    if tmk_fields:
                        print(f"  TMK fields found: {tmk_fields}")
                        
                        with arcpy.da.SearchCursor(shp_path, tmk_fields[:1]) as tmk_cursor:
                            sample_tmks = []
                            for i, row in enumerate(tmk_cursor):
                                if i < 5:
                                    sample_tmks.append(str(row[0]))
                                else:
                                    break
                            
                            print(f"  Sample TMKs: {sample_tmks}")
                            if sample_tmks:
                                print(f"  TMK length: {len(sample_tmks[0])} digits")
                    break  # Just check first shapefile for now
                    
            except Exception as e:
                print(f"  Error reading {shp}: {str(e)}")
    
    else:
        print("No shapefiles found in foundation folder")
        
else:
    print("❌ Foundation folder not found")

# Compare with bedroom data TMK format
print(f"\n--- BEDROOM DATA TMK FORMAT (FOR COMPARISON) ---")
gdb_path = r"C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\ParcelAnalysis.gdb"
bedroom_fc = os.path.join(gdb_path, "HI_Parcels_Bedrooms_Bathrooms_Units")

with arcpy.da.SearchCursor(bedroom_fc, ["TMK"]) as cursor:
    bedroom_tmks = []
    for i, row in enumerate(cursor):
        if i < 3:
            bedroom_tmks.append(str(row[0]))
        else:
            break

print(f"Bedroom data sample TMKs: {bedroom_tmks}")
print(f"Bedroom TMK length: {len(bedroom_tmks[0])} digits")

=== CHECKING FOUNDATION FOLDER ===
Foundation folder: C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\Outputs\foundation
Folder exists: True

Contents of foundation folder:
 1. Cesspool_Parcels_With_Attributes.cpg (0.0 MB)
 2. Cesspool_Parcels_With_Attributes.dbf (85.6 MB)
 3. Cesspool_Parcels_With_Attributes.prj (0.0 MB)
 4. Cesspool_Parcels_With_Attributes.sbn (0.6 MB)
 5. Cesspool_Parcels_With_Attributes.sbx (0.0 MB)
 6. Cesspool_Parcels_With_Attributes.shp (16.1 MB)
 7. Cesspool_Parcels_With_Attributes.shp.LEGIONOFBLOOM.14060.640.sr.lock (0.0 MB)
 8. Cesspool_Parcels_With_Attributes.shp.LEGIONOFBLOOM.14096.640.sr.lock (0.0 MB)
 9. Cesspool_Parcels_With_Attributes.shp.LEGIONOFBLOOM.20020.640.sr.lock (0.0 MB)
10. Cesspool_Parcels_With_Attributes.shp.LEGIONOFBLOOM.22728.640.sr.lock (0.0 MB)
11. Cesspool_Parcels_With_Attributes.shp.xml (0.0 MB)
12. Cesspool_Parcels_With_Attributes.shx (0.5 MB)
13. Cesspool_Parcel_Polygons_Spatial.cpg (0.0 MB)
14. Cesspool_Parcel_Polygons_

In [31]:
print(f"\n=== CREATING MASTER ANALYSIS TABLE ===")
print("Joining cesspool parcels with bedroom data...")

# Output path for joined data
output_fc = os.path.join(gdb_path, "Cesspool_Parcels_With_Bedrooms")

try:
    # Perform spatial join based on TMK
    arcpy.management.JoinField(
        cesspool_shp,           # Target (cesspool data)
        "TMK",                  # Target join field  
        bedroom_fc,             # Join table (bedroom data)
        "TMK",                  # Join field
        ["SUM_Bedrooms", "SUM_Bathrooms", "COUNT_Units", "GISAcres", "BR_per_AC"]  # Fields to transfer
    )
    
    print("✅ Join completed!")
    
    # Copy result to geodatabase
    arcpy.management.CopyFeatures(cesspool_shp, output_fc)
    
    # Verify results
    result_count = int(arcpy.management.GetCount(output_fc)[0])
    print(f"✅ Master table created: {result_count:,} records")
    
    # Check bedroom data integration
    with arcpy.da.SearchCursor(output_fc, ["TMK", "SUM_Bedrooms", "SUM_Bathrooms"]) as cursor:
        bedroom_matches = 0
        for i, row in enumerate(cursor):
            if row[1] is not None and row[1] > 0:
                bedroom_matches += 1
            if i >= 1000:
                break
        
    print(f"Sample check: {bedroom_matches}/1000 records have bedroom data")
    
except Exception as e:
    print(f"❌ Join failed: {str(e)}")


=== CREATING MASTER ANALYSIS TABLE ===
Joining cesspool parcels with bedroom data...
❌ Join failed: name 'cesspool_shp' is not defined


In [32]:
import os
import arcpy

print("=== CREATING MASTER ANALYSIS TABLE ===")

# Define paths
foundation_folder = r"C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\Outputs\foundation"
cesspool_shp = os.path.join(foundation_folder, "Cesspool_Parcels_With_Attributes.shp")
gdb_path = r"C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\ParcelAnalysis.gdb"
bedroom_fc = os.path.join(gdb_path, "HI_Parcels_Bedrooms_Bathrooms_Units")
output_fc = os.path.join(gdb_path, "Cesspool_Parcels_With_Bedrooms")

print(f"Cesspool data: {cesspool_shp}")
print(f"Bedroom data:  {bedroom_fc}")
print(f"Output:        {output_fc}")
print()

try:
    # First copy cesspool data to geodatabase
    temp_cesspool = os.path.join(gdb_path, "temp_cesspool_for_join")
    
    print("Step 1: Copying cesspool data to geodatabase...")
    if arcpy.Exists(temp_cesspool):
        arcpy.management.Delete(temp_cesspool)
    
    arcpy.management.CopyFeatures(cesspool_shp, temp_cesspool)
    cesspool_count = int(arcpy.management.GetCount(temp_cesspool)[0])
    print(f"✅ Copied {cesspool_count:,} cesspool records")
    
    # Perform the join
    print("Step 2: Joining with bedroom data...")
    arcpy.management.JoinField(
        temp_cesspool,          # Target (cesspool data in GDB)
        "TMK",                  # Target join field  
        bedroom_fc,             # Join table (bedroom data)
        "TMK",                  # Join field
        ["SUM_Bedrooms", "SUM_Bathrooms", "COUNT_Units", "GISAcres", "BR_per_AC"]
    )
    
    print("✅ Join completed!")
    
    # Copy to final output
    print("Step 3: Creating final master table...")
    if arcpy.Exists(output_fc):
        arcpy.management.Delete(output_fc)
    
    arcpy.management.CopyFeatures(temp_cesspool, output_fc)
    
    # Clean up temp data
    arcpy.management.Delete(temp_cesspool)
    
    # Verify results
    result_count = int(arcpy.management.GetCount(output_fc)[0])
    print(f"✅ Master table created: {result_count:,} records")
    
    # Analyze the join results
    print("\nStep 4: Analyzing join results...")
    with arcpy.da.SearchCursor(output_fc, ["TMK", "SUM_Bedrooms", "SUM_Bathrooms", "county"]) as cursor:
        total_checked = 0
        with_bedrooms = 0
        without_bedrooms = 0
        bedroom_distribution = {}
        counties = set()
        
        for row in cursor:
            total_checked += 1
            counties.add(row[3])
            
            if row[1] is not None and row[1] > 0:
                with_bedrooms += 1
                bedrooms = int(row[1])
                bedroom_distribution[bedrooms] = bedroom_distribution.get(bedrooms, 0) + 1
            else:
                without_bedrooms += 1
    
    print(f"\n--- JOIN ANALYSIS RESULTS ---")
    print(f"Total cesspool parcels: {total_checked:,}")
    print(f"With bedroom data: {with_bedrooms:,} ({with_bedrooms/total_checked*100:.1f}%)")
    print(f"Without bedroom data: {without_bedrooms:,} ({without_bedrooms/total_checked*100:.1f}%)")
    print(f"Counties represented: {sorted(counties)}")
    
    if bedroom_distribution:
        print(f"\nBedroom distribution (top 10):")
        sorted_bedrooms = sorted(bedroom_distribution.items(), key=lambda x: x[1], reverse=True)[:10]
        for bedrooms, count in sorted_bedrooms:
            print(f"  {bedrooms} bedrooms: {count:,} parcels")
    
    if with_bedrooms > 0:
        print(f"\n✅ SUCCESS! Master table ready for Matrix analysis")
        print(f"Dataset: {output_fc}")
    else:
        print(f"\n⚠️ WARNING: No bedroom data joined - check TMK compatibility")

except Exception as e:
    print(f"❌ Error: {str(e)}")
    

=== CREATING MASTER ANALYSIS TABLE ===
Cesspool data: C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\Outputs\foundation\Cesspool_Parcels_With_Attributes.shp
Bedroom data:  C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\ParcelAnalysis.gdb\HI_Parcels_Bedrooms_Bathrooms_Units
Output:        C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\ParcelAnalysis.gdb\Cesspool_Parcels_With_Bedrooms

Step 1: Copying cesspool data to geodatabase...
✅ Copied 68,565 cesspool records
Step 2: Joining with bedroom data...
✅ Join completed!
Step 3: Creating final master table...
✅ Master table created: 68,565 records

Step 4: Analyzing join results...

--- JOIN ANALYSIS RESULTS ---
Total cesspool parcels: 68,565
With bedroom data: 0 (0.0%)
Without bedroom data: 68,565 (100.0%)
Counties represented: ['Hawaii', 'Honolulu', 'Kauai', 'Maui']

⚠️ WARNING: No bedroom data joined - check TMK compatibility


In [33]:
print("=== DETAILED TMK FORMAT INVESTIGATION ===")

# Paths
foundation_folder = r"C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\Outputs\foundation"
cesspool_shp = os.path.join(foundation_folder, "Cesspool_Parcels_With_Attributes.shp")
gdb_path = r"C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\ParcelAnalysis.gdb"
bedroom_fc = os.path.join(gdb_path, "HI_Parcels_Bedrooms_Bathrooms_Units")

print("Collecting TMK samples from both datasets...")

# Get cesspool TMKs by county
cesspool_tmks_by_county = {}
with arcpy.da.SearchCursor(cesspool_shp, ["TMK", "county"]) as cursor:
    for i, row in enumerate(cursor):
        county = row[1]
        tmk = str(row[0])
        if county not in cesspool_tmks_by_county:
            cesspool_tmks_by_county[county] = []
        if len(cesspool_tmks_by_county[county]) < 5:
            cesspool_tmks_by_county[county].append(tmk)

print("--- CESSPOOL TMK SAMPLES BY COUNTY ---")
for county, tmks in cesspool_tmks_by_county.items():
    print(f"{county}: {tmks}")

# Get bedroom TMKs by county  
bedroom_tmks_by_county = {}
with arcpy.da.SearchCursor(bedroom_fc, ["TMK", "county"]) as cursor:
    for i, row in enumerate(cursor):
        county = row[1]
        tmk = str(row[0])
        if county not in bedroom_tmks_by_county:
            bedroom_tmks_by_county[county] = []
        if len(bedroom_tmks_by_county[county]) < 5:
            bedroom_tmks_by_county[county].append(tmk)

print("\n--- BEDROOM TMK SAMPLES BY COUNTY ---")
for county, tmks in bedroom_tmks_by_county.items():
    print(f"{county}: {tmks}")

# Compare patterns
print("\n--- TMK PATTERN ANALYSIS ---")
for county in ['Hawaii', 'Honolulu', 'Kauai', 'Maui']:
    if county in cesspool_tmks_by_county and county in bedroom_tmks_by_county:
        cp_sample = cesspool_tmks_by_county[county][0] if cesspool_tmks_by_county[county] else "None"
        br_sample = bedroom_tmks_by_county[county][0] if bedroom_tmks_by_county[county] else "None"
        
        print(f"\n{county} County:")
        print(f"  Cesspool: {cp_sample}")
        print(f"  Bedroom:  {br_sample}")
        
        if cp_sample != "None" and br_sample != "None":
            # Compare digit patterns
            if len(cp_sample) == len(br_sample):
                print(f"  Length match: ✅ Both {len(cp_sample)} digits")
                
                # Check if leading digits match (island codes)
                cp_prefix = cp_sample[:3] if len(cp_sample) >= 3 else cp_sample[:2]
                br_prefix = br_sample[:3] if len(br_sample) >= 3 else br_sample[:2]
                
                if cp_prefix == br_prefix:
                    print(f"  Prefix match: ✅ Both start with {cp_prefix}")
                else:
                    print(f"  Prefix mismatch: ❌ {cp_prefix} vs {br_prefix}")
            else:
                print(f"  Length mismatch: ❌ {len(cp_sample)} vs {len(br_sample)} digits")

# Test actual overlaps with full datasets
print(f"\n--- DIRECT TMK OVERLAP TEST ---")
print("Building TMK sets (this may take a moment)...")

cesspool_tmk_set = set()
with arcpy.da.SearchCursor(cesspool_shp, ["TMK"]) as cursor:
    for row in cursor:
        cesspool_tmk_set.add(str(row[0]))

print(f"Cesspool TMKs: {len(cesspool_tmk_set):,} unique values")

# Sample bedroom TMKs (test with smaller set first)
bedroom_tmk_sample = set()
with arcpy.da.SearchCursor(bedroom_fc, ["TMK"]) as cursor:
    for i, row in enumerate(cursor):
        bedroom_tmk_sample.add(str(row[0]))
        if i >= 50000:  # Test with 50k sample
            break

print(f"Bedroom TMK sample: {len(bedroom_tmk_sample):,} values")

overlap = cesspool_tmk_set.intersection(bedroom_tmk_sample)
print(f"Direct matches: {len(overlap):,}")

if len(overlap) > 0:
    print(f"Sample matches: {list(overlap)[:10]}")
else:
    print("❌ NO DIRECT MATCHES - TMK formats are incompatible")
    print("\nPossible issues:")
    print("1. Different TMK numbering systems between datasets")
    print("2. Leading zeros missing/added") 
    print("3. Different island/county coding systems")
    print("4. One dataset uses parcel numbers vs full TMKs")

=== DETAILED TMK FORMAT INVESTIGATION ===
--- CESSPOOL TMK SAMPLES BY COUNTY ---
Hawaii: ['389014053', '389014054', '389014055', '389014056', '388009127']
Maui: ['211003003', '211003012', '211003030', '211003031', '211003041']
Kauai: ['412006014', '412006041', '412008999', '412010002', '412010003']
Honolulu: ['111044013', '111065040', '111065041', '111065042', '111065046']

--- BEDROOM TMK SAMPLES BY COUNTY ---
Honolulu: ['111001001', '111002001', '111002002', '111002004', '111002006']
Maui: ['211001001', '211001002', '211001003', '211001004', '211001005']
Hawaii: ['311001001', '311001002', '311001003', '311001004', '311001006']
Kauai: ['411001001', '411001002', '411001003', '412001001', '412001003']

--- TMK PATTERN ANALYSIS ---

Hawaii County:
  Cesspool: 389014053
  Bedroom:  311001001
  Length match: ✅ Both 9 digits
  Prefix mismatch: ❌ 389 vs 311

Honolulu County:
  Cesspool: 111044013
  Bedroom:  111001001
  Length match: ✅ Both 9 digits
  Prefix match: ✅ Both start with 111

Kau

In [34]:
print("=== TMK CONVERSION AND RE-JOIN ===")

gdb_path = r"C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\ParcelAnalysis.gdb"
bedroom_fc = os.path.join(gdb_path, "HI_Parcels_Bedrooms_Bathrooms_Units")
cesspool_master = os.path.join(gdb_path, "Cesspool_Parcels_With_Bedrooms")

print("Creating TMK conversion mapping...")

# Create conversion lookup for problematic counties
tmk_conversion = {
    # Hawaii County: 389/388 -> 311
    '389': '311',
    '388': '311', 
    # Kauai County: 412 -> 411  
    '412': '411'
    # Honolulu (111) and Maui (211) don't need conversion
}

print("Step 1: Adding converted TMK field to bedroom data...")

# Add a new field for converted TMK
converted_tmk_field = "TMK_converted"
if converted_tmk_field not in [f.name for f in arcpy.ListFields(bedroom_fc)]:
    arcpy.management.AddField(bedroom_fc, converted_tmk_field, "TEXT", "", "", 20)

# Populate the converted TMK field
with arcpy.da.UpdateCursor(bedroom_fc, ["TMK", "county", converted_tmk_field]) as cursor:
    for row in cursor:
        original_tmk = str(row[0])
        county = row[1]
        
        # Convert TMK based on county and prefix
        if len(original_tmk) >= 3:
            prefix = original_tmk[:3]
            
            # Convert Hawaii County TMKs (311 -> 389)
            if county == "Hawaii" and prefix == "311":
                converted_tmk = "389" + original_tmk[3:]
            # Convert Kauai County TMKs (411 -> 412) 
            elif county == "Kauai" and prefix == "411":
                converted_tmk = "412" + original_tmk[3:]
            else:
                converted_tmk = original_tmk
        else:
            converted_tmk = original_tmk
            
        row[2] = converted_tmk
        cursor.updateRow(row)

print("✅ TMK conversion completed")

print("Step 2: Re-joining with converted TMKs...")

# Try the join again with converted TMKs
try:
    # Clear any existing bedroom fields from cesspool data
    cesspool_fields = [f.name for f in arcpy.ListFields(cesspool_master)]
    bedroom_fields_to_remove = [f for f in cesspool_fields if f in ["SUM_Bedrooms", "SUM_Bathrooms", "COUNT_Units", "GISAcres", "BR_per_AC"]]
    
    for field in bedroom_fields_to_remove:
        if field in cesspool_fields:
            try:
                arcpy.management.DeleteField(cesspool_master, field)
            except:
                pass
    
    # Perform join with converted TMKs
    arcpy.management.JoinField(
        cesspool_master,        # Target
        "TMK",                  # Target join field (cesspool TMKs)
        bedroom_fc,             # Join table  
        "TMK_converted",        # Join field (converted bedroom TMKs)
        ["SUM_Bedrooms", "SUM_Bathrooms", "COUNT_Units", "GISAcres", "BR_per_AC"]
    )
    
    print("✅ Re-join completed with converted TMKs!")
    
    # Analyze results
    print("Step 3: Analyzing improved join results...")
    
    with arcpy.da.SearchCursor(cesspool_master, ["TMK", "SUM_Bedrooms", "county"]) as cursor:
        total = 0
        with_bedrooms = 0
        by_county = {}
        
        for row in cursor:
            total += 1
            county = row[2]
            
            if county not in by_county:
                by_county[county] = {"total": 0, "with_bedrooms": 0}
            by_county[county]["total"] += 1
            
            if row[1] is not None and row[1] > 0:
                with_bedrooms += 1
                by_county[county]["with_bedrooms"] += 1
    
    print(f"\n--- IMPROVED JOIN RESULTS ---")
    print(f"Total cesspool parcels: {total:,}")
    print(f"With bedroom data: {with_bedrooms:,} ({with_bedrooms/total*100:.1f}%)")
    
    print(f"\nResults by county:")
    for county, stats in by_county.items():
        match_rate = stats["with_bedrooms"]/stats["total"]*100 if stats["total"] > 0 else 0
        print(f"  {county}: {stats['with_bedrooms']:,}/{stats['total']:,} ({match_rate:.1f}%)")
    
    if with_bedrooms > (total * 0.3):  # If >30% match rate
        print(f"\n✅ EXCELLENT! Master table now ready for Matrix analysis")
        print(f"Dataset: Cesspool_Parcels_With_Bedrooms")
        print(f"Ready to run technology compatibility assessment!")
    else:
        print(f"\n⚠️ Still low match rate - may need additional TMK investigation")

except Exception as e:
    print(f"❌ Re-join failed: {str(e)}")

=== TMK CONVERSION AND RE-JOIN ===
Creating TMK conversion mapping...
Step 1: Adding converted TMK field to bedroom data...
✅ TMK conversion completed
Step 2: Re-joining with converted TMKs...
✅ Re-join completed with converted TMKs!
Step 3: Analyzing improved join results...

--- IMPROVED JOIN RESULTS ---
Total cesspool parcels: 68,565
With bedroom data: 0 (0.0%)

Results by county:
  Hawaii: 0/42,037 (0.0%)
  Maui: 0/8,248 (0.0%)
  Kauai: 0/10,551 (0.0%)
  Honolulu: 0/7,729 (0.0%)

⚠️ Still low match rate - may need additional TMK investigation


In [35]:
print("=== DEBUGGING TMK CONVERSION ===")

gdb_path = r"C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\ParcelAnalysis.gdb"
bedroom_fc = os.path.join(gdb_path, "HI_Parcels_Bedrooms_Bathrooms_Units")

print("Checking what actually got converted...")

# Check the converted field values
with arcpy.da.SearchCursor(bedroom_fc, ["TMK", "county", "TMK_converted"]) as cursor:
    conversion_samples = {}
    for i, row in enumerate(cursor):
        county = row[1]
        if county not in conversion_samples:
            conversion_samples[county] = []
        if len(conversion_samples[county]) < 3:
            conversion_samples[county].append({
                "original": str(row[0]),
                "converted": str(row[2])
            })

print("--- CONVERSION RESULTS BY COUNTY ---")
for county, samples in conversion_samples.items():
    print(f"\n{county} County:")
    for sample in samples:
        print(f"  {sample['original']} → {sample['converted']}")

# Let's try a direct approach - create a lookup table instead
print("\n=== CREATING DIRECT TMK LOOKUP TABLE ===")

foundation_folder = r"C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\Outputs\foundation"
cesspool_shp = os.path.join(foundation_folder, "Cesspool_Parcels_With_Attributes.shp")

# Get all cesspool TMKs by county
cesspool_tmks_by_county = {}
with arcpy.da.SearchCursor(cesspool_shp, ["TMK", "county"]) as cursor:
    for row in cursor:
        county = row[1]
        tmk = str(row[0])
        if county not in cesspool_tmks_by_county:
            cesspool_tmks_by_county[county] = set()
        cesspool_tmks_by_county[county].add(tmk)

# Get bedroom TMKs by county  
bedroom_tmks_by_county = {}
with arcpy.da.SearchCursor(bedroom_fc, ["TMK", "county"]) as cursor:
    for row in cursor:
        county = row[1] 
        tmk = str(row[0])
        if county not in bedroom_tmks_by_county:
            bedroom_tmks_by_county[county] = set()
        bedroom_tmks_by_county[county].add(tmk)

print("--- TMK SET SIZES BY COUNTY ---")
for county in ['Hawaii', 'Maui', 'Kauai', 'Honolulu']:
    cp_count = len(cesspool_tmks_by_county.get(county, set()))
    br_count = len(bedroom_tmks_by_county.get(county, set()))
    
    if cp_count > 0 and br_count > 0:
        direct_overlap = cesspool_tmks_by_county[county].intersection(bedroom_tmks_by_county[county])
        print(f"\n{county}:")
        print(f"  Cesspool TMKs: {cp_count:,}")
        print(f"  Bedroom TMKs:  {br_count:,}") 
        print(f"  Direct overlap: {len(direct_overlap):,}")
        
        if len(direct_overlap) > 0:
            print(f"  Sample matches: {list(direct_overlap)[:5]}")

# Try alternative approach - use the other TMK fields
print(f"\n=== CHECKING OTHER TMK FIELDS ===")

# Check cesspool data for alternative TMK fields
cesspool_fields = [f.name for f in arcpy.ListFields(cesspool_shp)]
tmk_fields = [f for f in cesspool_fields if 'tmk' in f.lower()]
print(f"Cesspool TMK fields: {tmk_fields}")

if len(tmk_fields) > 1:
    print("Checking alternative TMK field values...")
    with arcpy.da.SearchCursor(cesspool_shp, tmk_fields[:4]) as cursor:  # Check first 4 TMK fields
        for i, row in enumerate(cursor):
            if i < 5:  # Show first 5 records
                print(f"  Record {i+1}: {dict(zip(tmk_fields[:4], row))}")

# Check bedroom data alternative TMK formats
print(f"\nChecking if bedroom data has county-specific TMK fields...")
bedroom_fields = [f.name for f in arcpy.ListFields(bedroom_fc)]
print(f"Available fields: {[f for f in bedroom_fields if 'tmk' in f.lower() or 'cty' in f.lower()]}")

=== DEBUGGING TMK CONVERSION ===
Checking what actually got converted...
--- CONVERSION RESULTS BY COUNTY ---

Honolulu County:
  111001001 → 111001001
  111002001 → 111002001
  111002002 → 111002002

Maui County:
  211001001 → 211001001
  211001002 → 211001002
  211001003 → 211001003

Hawaii County:
  311001001 → 389001001
  311001002 → 389001002
  311001003 → 389001003

Kauai County:
  411001001 → 412001001
  411001002 → 412001002
  411001003 → 412001003

=== CREATING DIRECT TMK LOOKUP TABLE ===
--- TMK SET SIZES BY COUNTY ---

Hawaii:
  Cesspool TMKs: 42,035
  Bedroom TMKs:  135,369
  Direct overlap: 42,025
  Sample matches: ['381006131', '381005024', '399007050', '399004016', '366007048']

Maui:
  Cesspool TMKs: 8,248
  Bedroom TMKs:  51,875
  Direct overlap: 8,248
  Sample matches: ['227006102', '221013062', '224037054', '221014117', '225004084']

Kauai:
  Cesspool TMKs: 10,551
  Bedroom TMKs:  25,068
  Direct overlap: 10,550
  Sample matches: ['428026054', '414004053', '418015035

In [36]:
print("=== FIXING THE JOIN WITH CORRECT TMK FIELD ===")

gdb_path = r"C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\ParcelAnalysis.gdb"
bedroom_fc = os.path.join(gdb_path, "HI_Parcels_Bedrooms_Bathrooms_Units")
cesspool_master = os.path.join(gdb_path, "Cesspool_Parcels_With_Bedrooms")

print("The direct overlap analysis shows:")
print("  Hawaii: 42,025/42,035 matches (99.98%)")
print("  Maui: 8,248/8,248 matches (100%)")  
print("  Kauai: 10,550/10,551 matches (99.99%)")
print("  Honolulu: 7,726/7,729 matches (99.96%)")
print("Total expected matches: ~68,549 out of 68,563")
print()

print("Step 1: Clearing previous join results...")

# Remove bedroom fields that may exist from failed previous joins
cesspool_fields = [f.name for f in arcpy.ListFields(cesspool_master)]
bedroom_fields_to_remove = ["SUM_Bedrooms", "SUM_Bathrooms", "COUNT_Units", "GISAcres", "BR_per_AC"]

for field in bedroom_fields_to_remove:
    if field in cesspool_fields:
        try:
            arcpy.management.DeleteField(cesspool_master, field)
            print(f"  Removed field: {field}")
        except:
            pass

print("Step 2: Performing join with ORIGINAL TMK fields...")

try:
    # Join using the original TMK field (not the converted one)
    arcpy.management.JoinField(
        cesspool_master,        # Target (cesspool data)
        "TMK",                  # Target join field
        bedroom_fc,             # Join table (bedroom data)
        "TMK",                  # Join field - USE ORIGINAL, NOT CONVERTED!
        ["SUM_Bedrooms", "SUM_Bathrooms", "COUNT_Units", "GISAcres", "BR_per_AC"]
    )
    
    print("✅ Join completed with original TMK fields!")
    
    # Analyze the results
    print("Step 3: Analyzing final results...")
    
    with arcpy.da.SearchCursor(cesspool_master, ["TMK", "SUM_Bedrooms", "SUM_Bathrooms", "county"]) as cursor:
        total = 0
        with_bedrooms = 0
        zero_bedrooms = 0
        by_county = {}
        bedroom_distribution = {}
        
        for row in cursor:
            total += 1
            county = row[3]
            bedrooms = row[1]
            
            if county not in by_county:
                by_county[county] = {"total": 0, "with_bedrooms": 0, "zero_bedrooms": 0}
            by_county[county]["total"] += 1
            
            if bedrooms is not None:
                if bedrooms > 0:
                    with_bedrooms += 1
                    by_county[county]["with_bedrooms"] += 1
                    bedroom_distribution[bedrooms] = bedroom_distribution.get(bedrooms, 0) + 1
                else:
                    zero_bedrooms += 1
                    by_county[county]["zero_bedrooms"] += 1
    
    print(f"\n--- FINAL JOIN RESULTS ---")
    print(f"Total cesspool parcels: {total:,}")
    print(f"With bedroom data >0: {with_bedrooms:,} ({with_bedrooms/total*100:.1f}%)")
    print(f"With zero bedrooms: {zero_bedrooms:,} ({zero_bedrooms/total*100:.1f}%)")
    print(f"No bedroom data: {total-with_bedrooms-zero_bedrooms:,}")
    
    print(f"\nResults by county:")
    for county, stats in by_county.items():
        match_rate = (stats["with_bedrooms"] + stats["zero_bedrooms"])/stats["total"]*100 if stats["total"] > 0 else 0
        positive_rate = stats["with_bedrooms"]/stats["total"]*100 if stats["total"] > 0 else 0
        print(f"  {county}: {stats['with_bedrooms']:,} positive + {stats['zero_bedrooms']:,} zero = {match_rate:.1f}% joined ({positive_rate:.1f}% with bedrooms)")
    
    if bedroom_distribution:
        print(f"\nBedroom distribution (top 10):")
        sorted_bedrooms = sorted(bedroom_distribution.items(), key=lambda x: x[1], reverse=True)[:10]
        for bedrooms, count in sorted_bedrooms:
            print(f"  {bedrooms} bedrooms: {count:,} parcels")
    
    total_joined = with_bedrooms + zero_bedrooms
    if total_joined > (total * 0.95):  # If >95% joined
        print(f"\n🎉 EXCELLENT SUCCESS!")
        print(f"Master table complete: {total_joined:,}/{total:,} parcels joined ({total_joined/total*100:.1f}%)")
        print(f"Dataset ready: Cesspool_Parcels_With_Bedrooms")
        print(f"✅ READY FOR TECHNOLOGY MATRIX ANALYSIS!")
    else:
        print(f"\n⚠️ Partial success: {total_joined:,} joined ({total_joined/total*100:.1f}%)")

except Exception as e:
    print(f"❌ Join failed: {str(e)}")

=== FIXING THE JOIN WITH CORRECT TMK FIELD ===
The direct overlap analysis shows:
  Hawaii: 42,025/42,035 matches (99.98%)
  Maui: 8,248/8,248 matches (100%)
  Kauai: 10,550/10,551 matches (99.99%)
  Honolulu: 7,726/7,729 matches (99.96%)
Total expected matches: ~68,549 out of 68,563

Step 1: Clearing previous join results...
  Removed field: SUM_Bedrooms
  Removed field: SUM_Bathrooms
  Removed field: COUNT_Units
  Removed field: GISAcres
  Removed field: BR_per_AC
Step 2: Performing join with ORIGINAL TMK fields...
✅ Join completed with original TMK fields!
Step 3: Analyzing final results...

--- FINAL JOIN RESULTS ---
Total cesspool parcels: 68,565
With bedroom data >0: 0 (0.0%)
With zero bedrooms: 0 (0.0%)
No bedroom data: 68,565

Results by county:
  Hawaii: 0 positive + 0 zero = 0.0% joined (0.0% with bedrooms)
  Maui: 0 positive + 0 zero = 0.0% joined (0.0% with bedrooms)
  Kauai: 0 positive + 0 zero = 0.0% joined (0.0% with bedrooms)
  Honolulu: 0 positive + 0 zero = 0.0% joine

In [37]:
print("=== INVESTIGATING TMK DATA TYPES ===")

gdb_path = r"C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\ParcelAnalysis.gdb"
bedroom_fc = os.path.join(gdb_path, "HI_Parcels_Bedrooms_Bathrooms_Units")
cesspool_master = os.path.join(gdb_path, "Cesspool_Parcels_With_Bedrooms")

# Check field types
print("--- FIELD TYPE ANALYSIS ---")
cesspool_fields = arcpy.ListFields(cesspool_master)
for field in cesspool_fields:
    if 'tmk' in field.name.lower():
        print(f"Cesspool {field.name}: {field.type}, Length: {field.length}")

bedroom_fields = arcpy.ListFields(bedroom_fc)
for field in bedroom_fields:
    if 'tmk' in field.name.lower():
        print(f"Bedroom {field.name}: {field.type}, Length: {field.length}")

# Check actual values to see if there are format differences
print("\n--- TMK VALUE COMPARISON ---")
print("Cesspool TMK samples:")
with arcpy.da.SearchCursor(cesspool_master, ["TMK"]) as cursor:
    for i, row in enumerate(cursor):
        if i < 5:
            tmk_val = row[0]
            print(f"  {tmk_val} (type: {type(tmk_val)}, length: {len(str(tmk_val))})")

print("\nBedroom TMK samples:")
with arcpy.da.SearchCursor(bedroom_fc, ["TMK"]) as cursor:
    for i, row in enumerate(cursor):
        if i < 5:
            tmk_val = row[0]
            print(f"  {tmk_val} (type: {type(tmk_val)}, length: {len(str(tmk_val))})")

print("\n=== MANUAL JOIN APPROACH ===")
print("Creating manual join using Python dictionary...")

# Build lookup dictionary from bedroom data
print("Step 1: Building bedroom data lookup...")
bedroom_lookup = {}
with arcpy.da.SearchCursor(bedroom_fc, ["TMK", "SUM_Bedrooms", "SUM_Bathrooms", "COUNT_Units", "GISAcres", "BR_per_AC"]) as cursor:
    count = 0
    for row in cursor:
        tmk = str(row[0])  # Convert to string for consistent matching
        bedroom_lookup[tmk] = {
            "SUM_Bedrooms": row[1],
            "SUM_Bathrooms": row[2], 
            "COUNT_Units": row[3],
            "GISAcres": row[4],
            "BR_per_AC": row[5]
        }
        count += 1
        if count % 50000 == 0:
            print(f"  Processed {count:,} bedroom records...")

print(f"✅ Bedroom lookup created: {len(bedroom_lookup):,} records")

# Add bedroom fields to cesspool data if they don't exist
print("Step 2: Adding bedroom fields to cesspool data...")
bedroom_fields_to_add = [
    ("SUM_Bedrooms", "LONG"),
    ("SUM_Bathrooms", "LONG"), 
    ("COUNT_Units", "LONG"),
    ("GISAcres", "DOUBLE"),
    ("BR_per_AC", "DOUBLE")
]

existing_fields = [f.name for f in arcpy.ListFields(cesspool_master)]
for field_name, field_type in bedroom_fields_to_add:
    if field_name not in existing_fields:
        arcpy.management.AddField(cesspool_master, field_name, field_type)
        print(f"  Added field: {field_name}")

# Perform manual update
print("Step 3: Manually updating cesspool records with bedroom data...")
matched_count = 0
total_count = 0

with arcpy.da.UpdateCursor(cesspool_master, ["TMK", "SUM_Bedrooms", "SUM_Bathrooms", "COUNT_Units", "GISAcres", "BR_per_AC"]) as cursor:
    for row in cursor:
        total_count += 1
        tmk = str(row[0])  # Convert to string
        
        if tmk in bedroom_lookup:
            # Update with bedroom data
            bedroom_data = bedroom_lookup[tmk]
            row[1] = bedroom_data["SUM_Bedrooms"]
            row[2] = bedroom_data["SUM_Bathrooms"] 
            row[3] = bedroom_data["COUNT_Units"]
            row[4] = bedroom_data["GISAcres"]
            row[5] = bedroom_data["BR_per_AC"]
            cursor.updateRow(row)
            matched_count += 1
        
        if total_count % 10000 == 0:
            print(f"  Processed {total_count:,} records, {matched_count:,} matched...")

print(f"\n--- MANUAL JOIN RESULTS ---")
print(f"Total cesspool records: {total_count:,}")
print(f"Successfully matched: {matched_count:,}")
print(f"Match rate: {matched_count/total_count*100:.1f}%")

if matched_count > (total_count * 0.95):
    print(f"\n🎉 MANUAL JOIN SUCCESS!")
    print(f"✅ Master table complete and ready for Matrix analysis!")
    print(f"Dataset: Cesspool_Parcels_With_Bedrooms")
else:
    print(f"\n⚠️ Lower than expected match rate - investigating further...")

=== INVESTIGATING TMK DATA TYPES ===
--- FIELD TYPE ANALYSIS ---
Cesspool TMK: Integer, Length: 4
Cesspool TMK_txt: String, Length: 9
Cesspool cty_tmk: String, Length: 8
Cesspool TMK_1: Integer, Length: 4
Bedroom TMK: String, Length: 20
Bedroom TMK_converted: String, Length: 20

--- TMK VALUE COMPARISON ---
Cesspool TMK samples:
  389014053 (type: <class 'int'>, length: 9)
  389014054 (type: <class 'int'>, length: 9)
  389014055 (type: <class 'int'>, length: 9)
  389014056 (type: <class 'int'>, length: 9)
  388009127 (type: <class 'int'>, length: 9)

Bedroom TMK samples:
  111001001 (type: <class 'str'>, length: 9)
  111002001 (type: <class 'str'>, length: 9)
  111002002 (type: <class 'str'>, length: 9)
  111002004 (type: <class 'str'>, length: 9)
  111002006 (type: <class 'str'>, length: 9)

=== MANUAL JOIN APPROACH ===
Creating manual join using Python dictionary...
Step 1: Building bedroom data lookup...
  Processed 50,000 bedroom records...
  Processed 100,000 bedroom records...
  

In [38]:
print("=== MASTER TABLE ANALYSIS ===")

gdb_path = r"C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\ParcelAnalysis.gdb"
cesspool_master = os.path.join(gdb_path, "Cesspool_Parcels_With_Bedrooms")

print("Analyzing bedroom distribution in cesspool parcels...")

with arcpy.da.SearchCursor(cesspool_master, ["TMK", "SUM_Bedrooms", "SUM_Bathrooms", "GISAcres", "county"]) as cursor:
    bedroom_stats = {}
    county_stats = {}
    total_analyzed = 0
    
    for row in cursor:
        total_analyzed += 1
        county = row[4]
        bedrooms = row[1] if row[1] is not None else 0
        acres = row[3] if row[3] is not None else 0
        
        # County statistics
        if county not in county_stats:
            county_stats[county] = {"total": 0, "with_bedrooms": 0, "avg_acres": 0}
        county_stats[county]["total"] += 1
        if bedrooms > 0:
            county_stats[county]["with_bedrooms"] += 1
        county_stats[county]["avg_acres"] += acres
        
        # Bedroom distribution
        bedroom_stats[bedrooms] = bedroom_stats.get(bedrooms, 0) + 1

print(f"--- CESSPOOL PARCEL BEDROOM ANALYSIS ---")
print(f"Total cesspool parcels analyzed: {total_analyzed:,}")

print(f"\nBedroom distribution:")
sorted_bedrooms = sorted(bedroom_stats.items())[:15]  # Show first 15 categories
for bedrooms, count in sorted_bedrooms:
    percentage = count/total_analyzed*100
    print(f"  {bedrooms:2} bedrooms: {count:7,} parcels ({percentage:5.1f}%)")

print(f"\nCounty breakdown:")
for county, stats in county_stats.items():
    avg_acres = stats["avg_acres"]/stats["total"] if stats["total"] > 0 else 0
    bedroom_rate = stats["with_bedrooms"]/stats["total"]*100 if stats["total"] > 0 else 0
    print(f"  {county:8}: {stats['total']:5,} parcels, {bedroom_rate:4.1f}% with bedrooms, avg {avg_acres:.2f} acres")

# Calculate wastewater flow estimates
print(f"\n--- WASTEWATER FLOW ESTIMATES ---")
residential_parcels = sum(count for bedrooms, count in bedroom_stats.items() if bedrooms > 0)
total_bedrooms = sum(bedrooms * count for bedrooms, count in bedroom_stats.items() if bedrooms > 0)
avg_bedrooms = total_bedrooms / residential_parcels if residential_parcels > 0 else 0

print(f"Residential parcels (bedrooms >0): {residential_parcels:,}")
print(f"Total bedrooms: {total_bedrooms:,}")
print(f"Average bedrooms per residential parcel: {avg_bedrooms:.2f}")

# Wastewater flow calculation (200 gal/bedroom/day per HAR 11-62)
total_flow_gpd = total_bedrooms * 200
print(f"Total estimated wastewater flow: {total_flow_gpd:,} gallons/day")
print(f"Average flow per residential parcel: {total_flow_gpd/residential_parcels:.0f} gallons/day")

print(f"\n🎯 READY FOR TECHNOLOGY MATRIX!")
print("Master table contains all data needed for:")
print("• Bedroom-based wastewater flow calculations")
print("• Lot size analysis (GISAcres field)")  
print("• County-specific technology assessment")
print("• HAR 11-62 compliance evaluation")

=== MASTER TABLE ANALYSIS ===
Analyzing bedroom distribution in cesspool parcels...
--- CESSPOOL PARCEL BEDROOM ANALYSIS ---
Total cesspool parcels analyzed: 68,565

Bedroom distribution:
   0 bedrooms:   2,244 parcels (  3.3%)
   1 bedrooms:   2,693 parcels (  3.9%)
   2 bedrooms:   9,100 parcels ( 13.3%)
   3 bedrooms:  33,575 parcels ( 49.0%)
   4 bedrooms:  10,577 parcels ( 15.4%)
   5 bedrooms:   4,957 parcels (  7.2%)
   6 bedrooms:   2,624 parcels (  3.8%)
   7 bedrooms:   1,013 parcels (  1.5%)
   8 bedrooms:     545 parcels (  0.8%)
   9 bedrooms:     300 parcels (  0.4%)
  10 bedrooms:     183 parcels (  0.3%)
  11 bedrooms:      96 parcels (  0.1%)
  12 bedrooms:     117 parcels (  0.2%)
  13 bedrooms:      55 parcels (  0.1%)
  14 bedrooms:      38 parcels (  0.1%)

County breakdown:
  Hawaii  : 42,037 parcels, 95.5% with bedrooms, avg 9.52 acres
  Maui    : 8,248 parcels, 97.5% with bedrooms, avg 6.56 acres
  Kauai   : 10,551 parcels, 99.2% with bedrooms, avg 6.24 acres
  

In [39]:
print("=== CHECKING TMK_Master_Attributes STATUS ===")

gdb_path = r"C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\ParcelAnalysis.gdb"
tmk_master = os.path.join(gdb_path, "TMK_Master_Attributes")

print(f"Checking: {tmk_master}")
print(f"Exists in geodatabase: {arcpy.Exists(tmk_master)}")

if arcpy.Exists(tmk_master):
    # Analyze TMK_Master_Attributes
    count = int(arcpy.management.GetCount(tmk_master)[0])
    print(f"✅ TMK_Master_Attributes found: {count:,} records")
    
    # Check all fields
    fields = [f.name for f in arcpy.ListFields(tmk_master)]
    print(f"\nAll fields in TMK_Master_Attributes ({len(fields)} total):")
    for i, field in enumerate(fields, 1):
        print(f"{i:2}. {field}")
    
    # Categorize fields
    categories = {
        "TMK/ID": [f for f in fields if any(x in f.lower() for x in ['tmk', 'objectid', 'fid'])],
        "Location": [f for f in fields if any(x in f.lower() for x in ['county', 'island', 'zone'])],
        "Wells": [f for f in fields if any(x in f.lower() for x in ['well', 'distance', 'municipal', 'domestic'])],
        "Bedrooms": [f for f in fields if any(x in f.lower() for x in ['bedroom', 'bathroom', 'unit'])],
        "Spatial": [f for f in fields if any(x in f.lower() for x in ['slope', 'soil', 'area', 'acres', 'gis'])],
        "Regulatory": [f for f in fields if any(x in f.lower() for x in ['sma', 'flood', 'setback', 'regulatory'])]
    }
    
    print(f"\n--- FIELD CATEGORIES IN TMK_Master_Attributes ---")
    for category, cat_fields in categories.items():
        if cat_fields:
            print(f"{category}: {', '.join(cat_fields)}")
        else:
            print(f"{category}: ❌ MISSING")
    
    # Sample the data
    print(f"\n--- SAMPLE DATA ---")
    with arcpy.da.SearchCursor(tmk_master, fields[:10]) as cursor:  # First 10 fields
        field_names = fields[:10]
        print(f"Sample fields: {', '.join(field_names)}")
        for i, row in enumerate(cursor):
            if i < 3:
                print(f"Record {i+1}: {dict(zip(field_names, row))}")
    
    # Check if it needs bedroom data
    has_bedrooms = any('bedroom' in f.lower() for f in fields)
    if not has_bedrooms:
        print(f"\n⚠️ TMK_Master_Attributes MISSING bedroom data")
        print("Need to join bedroom data from Cesspool_Parcels_With_Bedrooms")
    else:
        print(f"\n✅ TMK_Master_Attributes already has bedroom data")

else:
    print("❌ TMK_Master_Attributes not found in geodatabase")
    print("\nChecking foundation folder...")
    
    foundation_folder = r"C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\Outputs\foundation"
    tmk_master_shp = os.path.join(foundation_folder, "TMK_Master_Attributes.shp")
    
    if os.path.exists(tmk_master_shp):
        print(f"✅ Found in foundation folder: {tmk_master_shp}")
        
        count = int(arcpy.management.GetCount(tmk_master_shp)[0])
        print(f"Records: {count:,}")
        
        print("Should we import this to the geodatabase?")
    else:
        print("❌ TMK_Master_Attributes not found anywhere")

print(f"\n=== NEXT STEPS ===")
print("1. Confirm TMK_Master_Attributes location")
print("2. Check if it has bedroom data")
print("3. If missing bedroom data, join from Cesspool_Parcels_With_Bedrooms")
print("4. Verify all Matrix analysis fields are present")

=== CHECKING TMK_Master_Attributes STATUS ===
Checking: C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\ParcelAnalysis.gdb\TMK_Master_Attributes
Exists in geodatabase: False
❌ TMK_Master_Attributes not found in geodatabase

Checking foundation folder...
✅ Found in foundation folder: C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\Outputs\foundation\TMK_Master_Attributes.shp
Records: 82,141
Should we import this to the geodatabase?

=== NEXT STEPS ===
1. Confirm TMK_Master_Attributes location
2. Check if it has bedroom data
3. If missing bedroom data, join from Cesspool_Parcels_With_Bedrooms
4. Verify all Matrix analysis fields are present


In [40]:
print("=== CLEANING UP AND ORGANIZING GEODATABASE ===")
print("Importing all foundation outputs and creating complete master table")
print()

import os
import arcpy

# Define paths
gdb_path = r"C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\ParcelAnalysis.gdb"
foundation_folder = r"C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\Outputs\foundation"

print("Step 1: Inventory foundation folder shapefiles...")

# List all shapefiles in foundation folder
foundation_files = []
if os.path.exists(foundation_folder):
    for file in os.listdir(foundation_folder):
        if file.endswith('.shp'):
            shp_path = os.path.join(foundation_folder, file)
            try:
                count = int(arcpy.management.GetCount(shp_path)[0])
                foundation_files.append({
                    'name': file[:-4],  # Remove .shp extension
                    'path': shp_path,
                    'count': count
                })
            except:
                print(f"  ⚠️ Could not read {file}")

    print("Foundation shapefiles found:")
    for i, file_info in enumerate(foundation_files, 1):
        print(f"  {i}. {file_info['name']} ({file_info['count']:,} records)")

print(f"\nStep 2: Import key foundation files to geodatabase...")

# Priority imports - the most important files
priority_imports = [
    'TMK_Master_Attributes',
    'Cesspool_Parcels_With_Attributes', 
    'Cesspool_Parcel_Polygons_Spatial'
]

imported_datasets = []
for priority in priority_imports:
    foundation_file = next((f for f in foundation_files if f['name'] == priority), None)
    if foundation_file:
        print(f"\nImporting {priority}...")
        
        source_path = foundation_file['path']
        target_path = os.path.join(gdb_path, priority)
        
        try:
            # Remove if already exists
            if arcpy.Exists(target_path):
                arcpy.management.Delete(target_path)
                print(f"  Removed existing {priority}")
            
            # Import to geodatabase
            arcpy.management.CopyFeatures(source_path, target_path)
            
            # Verify
            count = int(arcpy.management.GetCount(target_path)[0])
            fields = [f.name for f in arcpy.ListFields(target_path)]
            
            print(f"  ✅ Imported {count:,} records")
            print(f"  Fields: {len(fields)} total")
            
            imported_datasets.append({
                'name': priority,
                'path': target_path,
                'count': count,
                'fields': fields
            })
            
        except Exception as e:
            print(f"  ❌ Import failed: {str(e)}")
    else:
        print(f"  ⚠️ {priority} not found in foundation folder")

print(f"\nStep 3: Analyze imported datasets...")

for dataset in imported_datasets:
    print(f"\n--- {dataset['name']} ANALYSIS ---")
    print(f"Records: {dataset['count']:,}")
    
    # Categorize fields
    fields = dataset['fields']
    categories = {
        "TMK/ID": [f for f in fields if any(x in f.lower() for x in ['tmk', 'objectid', 'fid'])],
        "Location": [f for f in fields if any(x in f.lower() for x in ['county', 'island', 'zone'])],
        "Wells": [f for f in fields if any(x in f.lower() for x in ['well', 'distance', 'municipal', 'domestic'])],
        "Bedrooms": [f for f in fields if any(x in f.lower() for x in ['bedroom', 'bathroom', 'unit'])],
        "Spatial": [f for f in fields if any(x in f.lower() for x in ['slope', 'soil', 'area', 'acres'])],
        "Matrix": [f for f in fields if any(x in f.lower() for x in ['matrix', 'compatible', 'suitable'])]
    }
    
    for category, cat_fields in categories.items():
        if cat_fields:
            print(f"  {category}: {', '.join(cat_fields[:5])}{'...' if len(cat_fields) > 5 else ''}")

# Now check our bedroom data 
bedroom_data = os.path.join(gdb_path, "Cesspool_Parcels_With_Bedrooms")
if arcpy.Exists(bedroom_data):
    print(f"\n--- BEDROOM DATA STATUS ---")
    count = int(arcpy.management.GetCount(bedroom_data)[0])
    print(f"Cesspool_Parcels_With_Bedrooms: {count:,} records")
    
    bedroom_fields = [f.name for f in arcpy.ListFields(bedroom_data)]
    bedroom_specific = [f for f in bedroom_fields if any(x in f.lower() for x in ['bedroom', 'bathroom', 'unit', 'gis'])]
    print(f"Bedroom fields: {', '.join(bedroom_specific)}")

print(f"\n=== NEXT STEP: CREATE COMPLETE MASTER TABLE ===")
print("Options:")
print("A. Use TMK_Master_Attributes as base (if it has well distances)")
print("B. Use Cesspool_Parcels_With_Bedrooms as base (has bedroom data)")
print("C. Create new master by joining all components")
print("\nWhich approach should we take?")

=== CLEANING UP AND ORGANIZING GEODATABASE ===
Importing all foundation outputs and creating complete master table

Step 1: Inventory foundation folder shapefiles...
Foundation shapefiles found:
  1. Cesspool_Parcels_With_Attributes (68,565 records)
  2. Cesspool_Parcel_Polygons_Spatial (68,565 records)
  3. Maui_Cesspool_Parcels_Final (0 records)
  4. Maui_Cesspool_Parcels_Pilot (8,248 records)
  5. TMK_Master_Attributes (82,141 records)

Step 2: Import key foundation files to geodatabase...

Importing TMK_Master_Attributes...
  ✅ Imported 82,141 records
  Fields: 18 total

Importing Cesspool_Parcels_With_Attributes...
  ✅ Imported 68,565 records
  Fields: 38 total

Importing Cesspool_Parcel_Polygons_Spatial...
  ✅ Imported 68,565 records
  Fields: 20 total

Step 3: Analyze imported datasets...

--- TMK_Master_Attributes ANALYSIS ---
Records: 82,141
  TMK/ID: OBJECTID, TMK
  Location: Island
  Spatial: SLOPE_PCT, SOIL_PERC, SOIL_KSAT, AVAIL_AREA

--- Cesspool_Parcels_With_Attributes A

In [41]:
print("=== CREATING MASTER TABLE WITH FULL SPATIAL COVERAGE ===")
print("Base: TMK_Master_Attributes (82,141 records)")
print("Join: Cesspool bedroom data (68,565 records)")
print("Result: Complete coverage with cesspool data where applicable")
print()

gdb_path = r"C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\ParcelAnalysis.gdb"

# Define data sources
base_table = os.path.join(gdb_path, "TMK_Master_Attributes")  # 82k records - our base
bedroom_source = os.path.join(gdb_path, "Cesspool_Parcels_With_Bedrooms")  # 68k records - join data
output_master = os.path.join(gdb_path, "Complete_Master_Analysis_Table")

print("Step 1: Using TMK_Master_Attributes as comprehensive base...")

# Check what we have in the base table
base_fields = [f.name for f in arcpy.ListFields(base_table)]
base_count = int(arcpy.management.GetCount(base_table)[0])

print(f"Base table: {base_count:,} records")
print(f"Base fields: {len(base_fields)} total")

# Categorize base fields
base_categories = {
    "Wells": [f for f in base_fields if any(x in f.lower() for x in ['well', 'distance', 'municipal', 'domestic'])],
    "Spatial": [f for f in base_fields if any(x in f.lower() for x in ['slope', 'soil', 'area', 'avail'])],
    "Location": [f for f in base_fields if any(x in f.lower() for x in ['county', 'island', 'zone'])]
}

print("Base table data categories:")
for category, fields in base_categories.items():
    if fields:
        print(f"  ✅ {category}: {', '.join(fields[:4])}{'...' if len(fields) > 4 else ''}")
    else:
        print(f"  ❌ {category}: Missing")

print(f"\nStep 2: Creating master table from base...")

try:
    # Create master table from TMK_Master_Attributes
    if arcpy.Exists(output_master):
        arcpy.management.Delete(output_master)
        print("  Removed existing master table")
    
    arcpy.management.CopyFeatures(base_table, output_master)
    master_count = int(arcpy.management.GetCount(output_master)[0])
    print(f"  ✅ Master table created: {master_count:,} records")
    
    print(f"\nStep 3: Adding bedroom data to all applicable parcels...")
    
    # Join bedroom data where TMKs match
    bedroom_fields_to_add = ["SUM_Bedrooms", "SUM_Bathrooms", "COUNT_Units", "county"]
    
    # Check which bedroom fields are available
    bedroom_fields = [f.name for f in arcpy.ListFields(bedroom_source)]
    available_fields = [f for f in bedroom_fields_to_add if f in bedroom_fields]
    
    print(f"  Adding bedroom fields: {', '.join(available_fields)}")
    
    arcpy.management.JoinField(
        output_master,          # Target (82k records)
        "TMK",                  # Target join field
        bedroom_source,         # Source (68k records with bedroom data)
        "TMK",                  # Source join field  
        available_fields        # Fields to add
    )
    
    print(f"  ✅ Bedroom data joined to master table")
    
    print(f"\nStep 4: Analyzing master table coverage...")
    
    # Analyze the join results
    with arcpy.da.SearchCursor(output_master, ["TMK", "SUM_Bedrooms", "county"]) as cursor:
        total_records = 0
        with_bedrooms = 0
        with_zero_bedrooms = 0
        cesspool_parcels = 0  # Records that matched cesspool data
        edge_cases = 0  # Records without cesspool data
        
        county_stats = {}
        
        for row in cursor:
            total_records += 1
            bedrooms = row[1]
            county = row[2]
            
            if county is not None:  # Matched with cesspool data
                cesspool_parcels += 1
                if county not in county_stats:
                    county_stats[county] = {"total": 0, "with_bedrooms": 0}
                county_stats[county]["total"] += 1
                
                if bedrooms is not None:
                    if bedrooms > 0:
                        with_bedrooms += 1
                        county_stats[county]["with_bedrooms"] += 1
                    else:
                        with_zero_bedrooms += 1
            else:
                edge_cases += 1
    
    print(f"--- MASTER TABLE ANALYSIS ---")
    print(f"Total records in master table: {total_records:,}")
    print(f"Cesspool parcels (matched): {cesspool_parcels:,} ({cesspool_parcels/total_records*100:.1f}%)")
    print(f"Edge cases (no cesspool data): {edge_cases:,} ({edge_cases/total_records*100:.1f}%)")
    print(f"")
    print(f"Of cesspool parcels:")
    print(f"  With bedrooms >0: {with_bedrooms:,} ({with_bedrooms/cesspool_parcels*100:.1f}%)")
    print(f"  With zero bedrooms: {with_zero_bedrooms:,} ({with_zero_bedrooms/cesspool_parcels*100:.1f}%)")
    
    if county_stats:
        print(f"\nBy county:")
        for county, stats in county_stats.items():
            bedroom_rate = stats["with_bedrooms"]/stats["total"]*100 if stats["total"] > 0 else 0
            print(f"  {county}: {stats['total']:,} parcels, {stats['with_bedrooms']:,} with bedrooms ({bedroom_rate:.1f}%)")
    
    # Check final field inventory
    final_fields = [f.name for f in arcpy.ListFields(output_master)]
    print(f"\nFinal master table: {len(final_fields)} fields total")
    
    # Final categorization
    final_categories = {
        "TMK/ID": [f for f in final_fields if any(x in f.lower() for x in ['tmk', 'objectid'])],
        "Location": [f for f in final_fields if any(x in f.lower() for x in ['county', 'island', 'zone'])],
        "Wells": [f for f in final_fields if any(x in f.lower() for x in ['well', 'distance', 'municipal', 'domestic'])],
        "Bedrooms": [f for f in final_fields if any(x in f.lower() for x in ['bedroom', 'bathroom', 'unit'])],
        "Spatial": [f for f in final_fields if any(x in f.lower() for x in ['slope', 'soil', 'area', 'avail'])],
    }
    
    print(f"\n--- COMPLETE MASTER TABLE CAPABILITIES ---")
    for category, cat_fields in final_categories.items():
        status = "✅" if cat_fields else "❌"
        field_count = len(cat_fields)
        sample_fields = ', '.join(cat_fields[:3]) + ('...' if len(cat_fields) > 3 else '')
        print(f"{status} {category} ({field_count}): {sample_fields if cat_fields else 'MISSING'}")
    
    if cesspool_parcels >= 68000 and with_bedrooms > 20000:
        print(f"\n🎉 MASTER TABLE SUCCESS!")
        print(f"✅ {total_records:,} total parcels with full spatial coverage")
        print(f"✅ {cesspool_parcels:,} cesspool parcels with bedroom data")
        print(f"✅ {edge_cases:,} edge case parcels for comprehensive analysis")
        print(f"✅ Ready for complete statewide technology matrix analysis!")
    else:
        print(f"\n⚠️ Lower coverage than expected - investigate data quality")

except Exception as e:
    print(f"❌ Master table creation failed: {str(e)}")

print(f"\n=== FINAL MASTER TABLE STATUS ===")
print(f"Dataset: Complete_Master_Analysis_Table")
print(f"Coverage: Statewide with cesspool focus")
print(f"Ready for Matrix: {'✅' if 'cesspool_parcels' in locals() and cesspool_parcels >= 68000 else '❌'}")

=== CREATING MASTER TABLE WITH FULL SPATIAL COVERAGE ===
Base: TMK_Master_Attributes (82,141 records)
Join: Cesspool bedroom data (68,565 records)
Result: Complete coverage with cesspool data where applicable

Step 1: Using TMK_Master_Attributes as comprehensive base...
Base table: 82,141 records
Base fields: 18 total
Base table data categories:
  ❌ Wells: Missing
  ✅ Spatial: SLOPE_PCT, SOIL_PERC, SOIL_KSAT, AVAIL_AREA
  ✅ Location: Island

Step 2: Creating master table from base...
  ✅ Master table created: 82,141 records

Step 3: Adding bedroom data to all applicable parcels...
  Adding bedroom fields: SUM_Bedrooms, SUM_Bathrooms, COUNT_Units, county
  ✅ Bedroom data joined to master table

Step 4: Analyzing master table coverage...
--- MASTER TABLE ANALYSIS ---
Total records in master table: 82,141
Cesspool parcels (matched): 79,155 (96.4%)
Edge cases (no cesspool data): 2,986 (3.6%)
﻿
Of cesspool parcels:
  With bedrooms >0: 78,469 (99.1%)
  With zero bedrooms: 686 (0.9%)

By coun

In [42]:
print("=== DIRECT WELLS DATA SEARCH AND FIX ===")
print("Searching geodatabase systematically for wells data")
print()

gdb_path = r"C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\ParcelAnalysis.gdb"
master_table = os.path.join(gdb_path, "Complete_Master_Analysis_Table")

# Set workspace to geodatabase
arcpy.env.workspace = gdb_path

print("Step 1: Complete geodatabase inventory...")

# List EVERYTHING in the geodatabase
all_feature_classes = arcpy.ListFeatureClasses()
all_tables = arcpy.ListTables()
all_datasets = arcpy.ListDatasets()

print(f"Feature classes ({len(all_feature_classes)}):")
wells_candidates = []
for i, fc in enumerate(all_feature_classes, 1):
    try:
        count = int(arcpy.management.GetCount(fc)[0])
        # Check for wells in name or fields
        fields = [f.name for f in arcpy.ListFields(fc)]
        has_wells = any('well' in fc.lower() for x in ['well']) or any('well' in f.lower() or 'distance' in f.lower() for f in fields)
        
        print(f"  {i:2}. {fc} ({count:,} records) {'🔍 WELLS?' if has_wells else ''}")
        
        if has_wells or 'well' in fc.lower():
            wells_candidates.append({
                'name': fc,
                'type': 'feature_class',
                'count': count,
                'fields': fields
            })
    except:
        print(f"  {i:2}. {fc} (error reading)")

if all_tables:
    print(f"\nTables ({len(all_tables)}):")
    for i, table in enumerate(all_tables, 1):
        try:
            count = int(arcpy.management.GetCount(table)[0])
            fields = [f.name for f in arcpy.ListFields(table)]
            has_wells = 'well' in table.lower() or any('well' in f.lower() for f in fields)
            
            print(f"  {i:2}. {table} ({count:,} records) {'🔍 WELLS?' if has_wells else ''}")
            
            if has_wells:
                wells_candidates.append({
                    'name': table,
                    'type': 'table', 
                    'count': count,
                    'fields': fields
                })
        except:
            print(f"  {i:2}. {table} (error reading)")

if all_datasets:
    print(f"\nFeature datasets ({len(all_datasets)}):")
    for dataset in all_datasets:
        print(f"  📁 {dataset}")
        
        # List contents of feature dataset
        arcpy.env.workspace = os.path.join(gdb_path, dataset)
        dataset_fcs = arcpy.ListFeatureClasses()
        
        for fc in dataset_fcs:
            try:
                count = int(arcpy.management.GetCount(fc)[0])
                full_path = os.path.join(gdb_path, dataset, fc)
                fields = [f.name for f in arcpy.ListFields(full_path)]
                has_wells = 'well' in fc.lower() or any('well' in f.lower() for f in fields)
                
                print(f"    └─ {fc} ({count:,} records) {'🔍 WELLS?' if has_wells else ''}")
                
                if has_wells:
                    wells_candidates.append({
                        'name': f"{dataset}/{fc}",
                        'full_path': full_path,
                        'type': 'feature_class_in_dataset',
                        'count': count,
                        'fields': fields
                    })
            except:
                print(f"    └─ {fc} (error reading)")

# Reset workspace
arcpy.env.workspace = gdb_path

print(f"\nStep 2: Wells data candidates found:")
if wells_candidates:
    for i, candidate in enumerate(wells_candidates, 1):
        well_fields = [f for f in candidate['fields'] if 'well' in f.lower() or 'distance' in f.lower()]
        print(f"  {i}. {candidate['name']} ({candidate['count']:,} records)")
        print(f"     Wells fields: {', '.join(well_fields)}")
        
    # Use the best candidate (most records with wells data)
    best_wells = max(wells_candidates, key=lambda x: len([f for f in x['fields'] if 'well' in f.lower()]))
    
    print(f"\nStep 3: Using best wells source: {best_wells['name']}")
    
    wells_source = best_wells.get('full_path') or os.path.join(gdb_path, best_wells['name'])
    wells_fields = [f for f in best_wells['fields'] if 'well' in f.lower() or 'distance' in f.lower()]
    
    print(f"Adding wells fields: {', '.join(wells_fields)}")
    
    try:
        arcpy.management.JoinField(
            master_table,       # Target
            "TMK",             # Target field
            wells_source,      # Wells source
            "TMK",             # Source field
            wells_fields       # Fields to add
        )
        
        print(f"✅ WELLS DATA ADDED TO MASTER TABLE!")
        
        # Verify
        master_fields = [f.name for f in arcpy.ListFields(master_table)]
        added_wells = [f for f in master_fields if 'well' in f.lower()]
        print(f"Wells fields now in master: {', '.join(added_wells)}")
        
    except Exception as e:
        print(f"❌ Wells join failed: {str(e)}")

else:
    print("❌ NO WELLS DATA FOUND ANYWHERE!")
    print("Check if wells data exists in a different location")

print(f"\n=== WELLS STATUS ===")
master_fields = [f.name for f in arcpy.ListFields(master_table)]
wells_in_master = [f for f in master_fields if 'well' in f.lower()]
print(f"Wells fields in master table: {wells_in_master if wells_in_master else 'NONE'}")
print(f"Status: {'✅ FIXED' if wells_in_master else '❌ STILL MISSING'}")

=== DIRECT WELLS DATA SEARCH AND FIX ===
Searching geodatabase systematically for wells data

Step 1: Complete geodatabase inventory...
Feature classes (11):
   1. Parcels_Hawaii (135,471 records) 
   2. tmk_state (384,262 records) 
   3. Parcels_Honolulu (171,905 records) 
   4. Parcels_Kauai (25,121 records) 
   5. Parcels_Maui (51,765 records) 
   6. HI_Parcels_Bedrooms_Bathrooms_Units (384,208 records) 
   7. Cesspool_Parcels_With_Bedrooms (68,565 records) 
   8. TMK_Master_Attributes (82,141 records) 
   9. Cesspool_Parcels_With_Attributes (68,565 records) 
  10. Cesspool_Parcel_Polygons_Spatial (68,565 records) 
  11. Complete_Master_Analysis_Table (82,141 records) 

Tables (7):
   1. ParcelsData_Maui_ExportTable (51,765 records) 
   2. bedrooms_out_ExportTable (70,322 records) 
   3. ParcelsData_Maui_ExportTable1 (51,765 records) 
   4. ParcelsData_Maui_ExportTable2 (51,765 records) 
   5. ParcelsData_Maui_ExportTable3 (51,765 records) 
   6. ParcelsData_Maui_ExportTable4 (51,76

In [43]:
print("=== IMPORTING WELLS DATA FROM CORRECT LOCATION ===")
print("Source: C:\\Users\\rober\\OneDrive\\Documents\\GIS_Projects\\ParcelAnalysis\\data\\gis_downloads\\wells\\statewide")
print()

import os
import arcpy

# Define paths
wells_folder = r"C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\data\gis_downloads\wells\statewide"
gdb_path = r"C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\ParcelAnalysis.gdb"
master_table = os.path.join(gdb_path, "Complete_Master_Analysis_Table")

print("Step 1: Inventory wells folder...")
print(f"Wells folder exists: {os.path.exists(wells_folder)}")

if os.path.exists(wells_folder):
    wells_files = []
    for file in os.listdir(wells_folder):
        file_path = os.path.join(wells_folder, file)
        if file.endswith('.shp'):
            try:
                count = int(arcpy.management.GetCount(file_path)[0])
                wells_files.append({
                    'name': file[:-4],  # Remove .shp
                    'path': file_path,
                    'count': count
                })
            except Exception as e:
                print(f"  Error reading {file}: {str(e)}")
    
    if wells_files:
        print("Wells files found:")
        for i, file_info in enumerate(wells_files, 1):
            print(f"  {i}. {file_info['name']} ({file_info['count']:,} records)")
            
            # Check fields in each wells file
            fields = [f.name for f in arcpy.ListFields(file_info['path'])]
            wells_fields = [f for f in fields if any(keyword in f.lower() for keyword in ['well', 'distance', 'municipal', 'domestic'])]
            if wells_fields:
                print(f"     Wells fields: {', '.join(wells_fields)}")
    
        print(f"\nStep 2: Import wells data to geodatabase...")
        
        # Import each wells file to geodatabase
        imported_wells = []
        for file_info in wells_files:
            target_name = f"Wells_{file_info['name']}"
            target_path = os.path.join(gdb_path, target_name)
            
            try:
                # Remove if exists
                if arcpy.Exists(target_path):
                    arcpy.management.Delete(target_path)
                
                # Import
                arcpy.management.CopyFeatures(file_info['path'], target_path)
                count = int(arcpy.management.GetCount(target_path)[0])
                
                print(f"  ✅ Imported {target_name}: {count:,} records")
                
                fields = [f.name for f in arcpy.ListFields(target_path)]
                wells_fields = [f for f in fields if any(keyword in f.lower() for keyword in ['well', 'distance', 'municipal', 'domestic', 'tmk'])]
                
                imported_wells.append({
                    'name': target_name,
                    'path': target_path,
                    'count': count,
                    'fields': fields,
                    'wells_fields': wells_fields
                })
                
            except Exception as e:
                print(f"  ❌ Failed to import {file_info['name']}: {str(e)}")
        
        if imported_wells:
            print(f"\nStep 3: Join wells data to master table...")
            
            # Use the wells dataset with the most records or best field coverage
            best_wells = max(imported_wells, key=lambda x: len(x['wells_fields']))
            
            print(f"Using wells source: {best_wells['name']}")
            print(f"Wells fields to join: {', '.join(best_wells['wells_fields'])}")
            
            # Check if wells data has TMK field for joining
            tmk_fields = [f for f in best_wells['fields'] if 'tmk' in f.lower()]
            if tmk_fields:
                join_field = tmk_fields[0]
                print(f"Join field: {join_field}")
                
                # Get only the distance/wells fields (not TMK)
                distance_fields = [f for f in best_wells['wells_fields'] if 'tmk' not in f.lower()]
                
                try:
                    arcpy.management.JoinField(
                        master_table,           # Target (master table)
                        "TMK",                  # Target join field
                        best_wells['path'],     # Source (wells data)
                        join_field,             # Source join field
                        distance_fields         # Fields to add (distances only)
                    )
                    
                    print(f"✅ WELLS DATA SUCCESSFULLY JOINED TO MASTER TABLE!")
                    
                    # Verify the join
                    master_fields = [f.name for f in arcpy.ListFields(master_table)]
                    added_wells_fields = [f for f in master_fields if any(keyword in f.lower() for keyword in ['well', 'distance', 'municipal', 'domestic'])]
                    
                    print(f"Wells fields now in master table: {', '.join(added_wells_fields)}")
                    
                    # Test data coverage
                    if added_wells_fields:
                        with arcpy.da.SearchCursor(master_table, ["TMK"] + added_wells_fields[:2]) as cursor:
                            sample_count = 0
                            wells_data_count = 0
                            
                            for row in cursor:
                                sample_count += 1
                                if len(row) > 1 and row[1] is not None:
                                    wells_data_count += 1
                                if sample_count >= 10000:
                                    break
                        
                        coverage = wells_data_count/sample_count*100 if sample_count > 0 else 0
                        print(f"Wells data coverage: {wells_data_count:,}/{sample_count:,} ({coverage:.1f}%)")
                        
                        if coverage > 70:
                            print(f"\n🎉 WELLS DATA INTEGRATION SUCCESS!")
                            print(f"✅ Master table now complete with wells distances")
                            print(f"✅ HAR 11-62 compliance ready (1000ft setbacks)")
                            print(f"✅ Technology Matrix fully supported!")
                        else:
                            print(f"\n⚠️ Lower wells coverage than expected")
                    
                except Exception as e:
                    print(f"❌ Wells join failed: {str(e)}")
                    
            else:
                print(f"❌ No TMK field found in wells data for joining")
                print(f"Available fields: {', '.join(best_wells['fields'])}")
        
    else:
        print("❌ No shapefile wells data found in wells folder")

else:
    print("❌ Wells folder not found!")
    print("Please verify the path is correct")

print(f"\n=== FINAL WELLS STATUS ===")
if arcpy.Exists(master_table):
    master_fields = [f.name for f in arcpy.ListFields(master_table)]
    wells_fields = [f for f in master_fields if any(keyword in f.lower() for keyword in ['well', 'distance', 'municipal', 'domestic'])]
    
    if wells_fields:
        print(f"✅ Wells data in master table: {', '.join(wells_fields)}")
        print(f"✅ Master table complete and ready for Matrix analysis!")
    else:
        print(f"❌ Wells data still missing from master table")
else:
    print(f"❌ Master table not found")

=== IMPORTING WELLS DATA FROM CORRECT LOCATION ===
Source: C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\data\gis_downloads\wells\statewide

Step 1: Inventory wells folder...
Wells folder exists: True
Wells files found:
  1. CPs_Distance_to_Domestic_Wells (82,141 records)
  2. CPs_Distance_to_Municipal_Wells (82,141 records)

Step 2: Import wells data to geodatabase...
  ✅ Imported Wells_CPs_Distance_to_Domestic_Wells: 82,141 records
  ✅ Imported Wells_CPs_Distance_to_Municipal_Wells: 82,141 records

Step 3: Join wells data to master table...
Using wells source: Wells_CPs_Distance_to_Domestic_Wells
Wells fields to join: TMK
Join field: TMK
✅ WELLS DATA SUCCESSFULLY JOINED TO MASTER TABLE!
Wells fields now in master table: 

=== FINAL WELLS STATUS ===
❌ Wells data still missing from master table


In [44]:
print("=== DIAGNOSING WELLS DATA FIELD NAMES ===")
print("Checking what fields actually exist in the wells datasets")
print()

gdb_path = r"C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\ParcelAnalysis.gdb"
master_table = os.path.join(gdb_path, "Complete_Master_Analysis_Table")

# Check the imported wells datasets
wells_datasets = [
    "Wells_CPs_Distance_to_Domestic_Wells",
    "Wells_CPs_Distance_to_Municipal_Wells"
]

print("Examining actual field names in wells datasets...")

for wells_name in wells_datasets:
    wells_path = os.path.join(gdb_path, wells_name)
    if arcpy.Exists(wells_path):
        fields = [f.name for f in arcpy.ListFields(wells_path)]
        field_info = []
        
        for field in fields:
            field_obj = next(f for f in arcpy.ListFields(wells_path) if f.name == field)
            field_info.append(f"{field} ({field_obj.type})")
        
        print(f"\n{wells_name}:")
        print(f"  All fields: {', '.join(field_info)}")
        
        # The distance values are probably numeric fields that aren't TMK
        numeric_fields = [f.name for f in arcpy.ListFields(wells_path) 
                         if f.type in ['Double', 'Single', 'Integer', 'SmallInteger'] 
                         and f.name not in ['OBJECTID', 'TMK']]
        
        print(f"  Numeric fields (likely distances): {', '.join(numeric_fields)}")
        
        # Sample the data to see values
        if numeric_fields:
            print(f"  Sample values:")
            with arcpy.da.SearchCursor(wells_path, ["TMK"] + numeric_fields[:2]) as cursor:
                for i, row in enumerate(cursor):
                    if i < 3:
                        print(f"    TMK {row[0]}: {dict(zip(numeric_fields[:2], row[1:]))}")
                    else:
                        break

# Now join the correct fields
print(f"\nJoining the actual distance fields...")

for wells_name in wells_datasets:
    wells_path = os.path.join(gdb_path, wells_name)
    if arcpy.Exists(wells_path):
        # Get numeric fields that aren't system fields
        numeric_fields = [f.name for f in arcpy.ListFields(wells_path) 
                         if f.type in ['Double', 'Single', 'Integer', 'SmallInteger'] 
                         and f.name not in ['OBJECTID', 'TMK', 'Shape_Length', 'Shape_Area']]
        
        if numeric_fields:
            print(f"\nJoining from {wells_name}:")
            print(f"  Distance fields: {', '.join(numeric_fields)}")
            
            try:
                arcpy.management.JoinField(
                    master_table,       # Target
                    "TMK",             # Target field
                    wells_path,        # Source
                    "TMK",             # Source field
                    numeric_fields     # Actual distance fields
                )
                
                print(f"  ✅ Successfully joined {len(numeric_fields)} distance fields")
                
            except Exception as e:
                print(f"  ❌ Join failed: {str(e)}")

# Final verification
print(f"\n=== FINAL VERIFICATION ===")

master_fields = [f.name for f in arcpy.ListFields(master_table)]
distance_fields = [f for f in master_fields if any(keyword in f.lower() 
                  for keyword in ['distance', 'dist', 'near', 'buffer', 'well', 'municipal', 'domestic'])]

print(f"Distance/wells fields now in master table: {', '.join(distance_fields)}")

if distance_fields:
    print(f"✅ WELLS DATA SUCCESSFULLY INTEGRATED!")
    print(f"✅ Master table ready for HAR 11-62 analysis (1000ft setbacks)")
    print(f"✅ Technology Matrix fully supported!")
else:
    print(f"❌ Wells distance fields still not found")
    print("Need to examine the wells data structure more carefully")

=== DIAGNOSING WELLS DATA FIELD NAMES ===
Checking what fields actually exist in the wells datasets

Examining actual field names in wells datasets...

Wells_CPs_Distance_to_Domestic_Wells:
  All fields: OBJECTID (OID), Shape (Geometry), X (Double), Y (Double), Island (String), TMK (Integer), Uid (String), dist2_DomW (Double)
  Numeric fields (likely distances): X, Y, dist2_DomW
  Sample values:
    TMK 186006001: {'X': -158.150145493, 'Y': 21.4425027002}
    TMK 186006001: {'X': -158.150088955, 'Y': 21.4422976382}
    TMK 186006001: {'X': -158.150329285, 'Y': 21.4426010061}

Wells_CPs_Distance_to_Municipal_Wells:
  All fields: OBJECTID (OID), Shape (Geometry), X (Double), Y (Double), Island (String), TMK (Integer), Uid (String), dist2_MunW (Double)
  Numeric fields (likely distances): X, Y, dist2_MunW
  Sample values:
    TMK 186006001: {'X': -158.150145493, 'Y': 21.4425027002}
    TMK 186006001: {'X': -158.150088955, 'Y': 21.4422976382}
    TMK 186006001: {'X': -158.150329285, 'Y': 2

In [45]:
print("=== COMPREHENSIVE BUILDING FOOTPRINTS BY COUNTY ===")
print("Processing all counties with appropriate data sources")
print()

import arcpy
import os

# Define paths and settings
gdb_path = r"C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\ParcelAnalysis.gdb"
master_table = os.path.join(gdb_path, "Complete_Master_Analysis_Table")

# County-specific data sources and processing plan
county_sources = {
    "Honolulu": {
        "source_layer": "Building Footprints CCH",
        "island": "Oahu",
        "output_name": "Honolulu_Building_Footprints"
    },
    "Maui": {
        "source_layer": "Maui County Building Footprints circa 2023", 
        "island": "Maui",
        "output_name": "Maui_Building_Footprints"
    },
    "Hawaii": {
        "source_layer": "USA_Structures_B",
        "island": "Hawaii", 
        "output_name": "Hawaii_Building_Footprints"
    },
    "Kauai": {
        "source_layer": "USA_Structures_B",
        "island": "Kauai",
        "output_name": "Kauai_Building_Footprints"
    }
}

print("Step 1: Verify source layers and master table...")

# Get current map layers
aprx = arcpy.mp.ArcGISProject("CURRENT")
map_obj = aprx.listMaps()[0]
available_layers = {layer.name: layer for layer in map_obj.listLayers()}

print(f"Available source layers:")
for county, config in county_sources.items():
    source_available = config["source_layer"] in available_layers
    print(f"  {county}: {config['source_layer']} {'✅' if source_available else '❌'}")

# Check master table
if arcpy.Exists(master_table):
    parcel_count = int(arcpy.management.GetCount(master_table)[0])
    print(f"\nMaster table: {parcel_count:,} parcels ✅")
else:
    print(f"\nMaster table: Missing ❌")
    exit()

print(f"\nStep 2: Processing each county...")

county_results = {}

for county, config in county_sources.items():
    print(f"\n--- Processing {county} County ---")
    
    if config["source_layer"] not in available_layers:
        print(f"❌ Source layer not available: {config['source_layer']}")
        continue
    
    try:
        # Create county-specific parcel subset
        county_parcels = os.path.join(gdb_path, f"temp_{county}_parcels")
        
        # Select parcels for this county
        where_clause = f"county = '{county}'"
        arcpy.management.Select(master_table, county_parcels, where_clause)
        
        county_parcel_count = int(arcpy.management.GetCount(county_parcels)[0])
        print(f"  County parcels: {county_parcel_count:,}")
        
        if county_parcel_count == 0:
            print(f"  ⚠️ No parcels found for {county}")
            continue
        
        # Get source layer
        source_layer = available_layers[config["source_layer"]]
        
        # For USA_Structures_B, pre-filter by island to improve performance
        if config["source_layer"] == "USA_Structures_B":
            print(f"  Pre-filtering USA structures for {config['island']} island...")
            
            # Create island boundary from county parcels
            island_boundary = os.path.join(gdb_path, f"temp_{county}_boundary")
            arcpy.management.MinimumBoundingGeometry(
                county_parcels,
                island_boundary, 
                "CONVEX_HULL",
                "ALL"
            )
            
            # Select buildings within island boundary first
            arcpy.management.SelectLayerByLocation(
                source_layer,
                "INTERSECT", 
                island_boundary,
                selection_type="NEW_SELECTION"
            )
            
            island_building_count = int(arcpy.management.GetCount(source_layer)[0])
            print(f"  Buildings in {config['island']}: {island_building_count:,}")
            
            # Clean up boundary
            arcpy.management.Delete(island_boundary)
        
        # Select buildings that intersect (touch or overlap) county parcels
        print(f"  Selecting buildings intersecting {county} parcels...")
        
        arcpy.management.SelectLayerByLocation(
            source_layer,
            "INTERSECT",
            county_parcels,
            selection_type="SUBSET_SELECTION" if config["source_layer"] == "USA_Structures_B" else "NEW_SELECTION"
        )
        
        selected_count = int(arcpy.management.GetCount(source_layer)[0])
        print(f"  Selected buildings: {selected_count:,}")
        
        if selected_count > 0:
            # Export buildings
            output_path = os.path.join(gdb_path, config["output_name"])
            
            if arcpy.Exists(output_path):
                arcpy.management.Delete(output_path)
            
            arcpy.management.CopyFeatures(source_layer, output_path)
            
            # Add TMK assignment field
            arcpy.management.AddField(output_path, "Assigned_TMK", "TEXT", "", "", 20)
            
            print(f"  Exported: {config['output_name']}")
            
            # Assign each building to the parcel containing most of its area
            print(f"  Assigning buildings to TMKs...")
            
            # Spatial join to find overlapping parcels for each building
            join_table = os.path.join(gdb_path, f"temp_{county}_building_parcel_join")
            
            arcpy.analysis.SpatialJoin(
                output_path,           # Target (buildings)
                county_parcels,        # Join (parcels) 
                join_table,
                "JOIN_ONE_TO_MANY",    # Multiple parcels per building
                "KEEP_ALL",
                match_option="INTERSECT"
            )
            
            # Calculate area of intersection for each building-parcel pair
            arcpy.management.AddField(join_table, "Overlap_Area", "DOUBLE")
            
            # Use geometry to calculate actual overlap area
            with arcpy.da.UpdateCursor(join_table, ["SHAPE@", "Overlap_Area"]) as cursor:
                for row in cursor:
                    if row[0]:
                        row[1] = row[0].area
                        cursor.updateRow(row)
            
            # Find TMK with maximum overlap for each building
            building_tmk_dict = {}
            with arcpy.da.SearchCursor(join_table, ["TARGET_FID", "TMK", "Overlap_Area"]) as cursor:
                for row in cursor:
                    building_fid = row[0]
                    tmk = row[1] 
                    area = row[2]
                    
                    if building_fid not in building_tmk_dict or area > building_tmk_dict[building_fid][1]:
                        building_tmk_dict[building_fid] = (tmk, area)
            
            # Update buildings with assigned TMK
            with arcpy.da.UpdateCursor(output_path, ["OBJECTID", "Assigned_TMK"]) as cursor:
                for row in cursor:
                    building_fid = row[0]
                    if building_fid in building_tmk_dict:
                        row[1] = str(building_tmk_dict[building_fid][0])
                        cursor.updateRow(row)
            
            # Calculate building areas and summarize by TMK
            arcpy.management.AddField(output_path, "Building_Area_SF", "DOUBLE")
            
            with arcpy.da.UpdateCursor(output_path, ["SHAPE@AREA", "Building_Area_SF"]) as cursor:
                for row in cursor:
                    row[1] = row[0]  # Area in square feet
                    cursor.updateRow(row)
            
            print(f"  ✅ TMK assignment complete")
            
            # Store results
            final_count = int(arcpy.management.GetCount(output_path)[0])
            county_results[county] = {
                "output": config["output_name"],
                "parcels": county_parcel_count,
                "buildings": final_count
            }
            
            # Clean up temporary files
            arcpy.management.Delete(join_table)
            
        else:
            print(f"  ⚠️ No buildings found for {county}")
        
        # Clear selection and clean up
        arcpy.management.SelectLayerByAttribute(source_layer, "CLEAR_SELECTION")
        arcpy.management.Delete(county_parcels)
        
    except Exception as e:
        print(f"  ❌ Error processing {county}: {str(e)}")

print(f"\nStep 3: Creating building area summary for master table...")

# Add building area field to master table
if "Area_Under_Structures" not in [f.name for f in arcpy.ListFields(master_table)]:
    arcpy.management.AddField(master_table, "Area_Under_Structures", "DOUBLE")

# Initialize all values to 0
with arcpy.da.UpdateCursor(master_table, ["Area_Under_Structures"]) as cursor:
    for row in cursor:
        row[0] = 0.0
        cursor.updateRow(row)

# Sum building areas by TMK for each county
total_buildings_processed = 0

for county, result in county_results.items():
    if result:
        building_layer = os.path.join(gdb_path, result["output"])
        
        print(f"  Processing {county} building areas...")
        
        # Create summary statistics
        tmk_building_areas = {}
        with arcpy.da.SearchCursor(building_layer, ["Assigned_TMK", "Building_Area_SF"]) as cursor:
            for row in cursor:
                tmk = row[0]
                area = row[1] if row[1] else 0
                
                if tmk and tmk != "None":
                    tmk_building_areas[tmk] = tmk_building_areas.get(tmk, 0) + area
                    total_buildings_processed += 1
        
        # Update master table with building areas
        with arcpy.da.UpdateCursor(master_table, ["TMK", "Area_Under_Structures", "county"]) as cursor:
            for row in cursor:
                if row[2] == county and str(row[0]) in tmk_building_areas:
                    row[1] = tmk_building_areas[str(row[0])]
                    cursor.updateRow(row)
        
        print(f"    {len(tmk_building_areas)} TMKs with buildings")

print(f"\n=== BUILDING FOOTPRINTS PROCESSING COMPLETE ===")
print(f"County results:")
for county, result in county_results.items():
    if result:
        print(f"  {county}: {result['buildings']:,} buildings across {result['parcels']:,} parcels")
        print(f"           Dataset: {result['output']}")

print(f"\nMaster table updated:")
print(f"  Total buildings processed: {total_buildings_processed:,}")
print(f"  Field added: Area_Under_Structures")

# Verify master table updates
with arcpy.da.SearchCursor(master_table, ["TMK", "Area_Under_Structures", "county"]) as cursor:
    county_stats = {}
    for row in cursor:
        county = row[2]
        area = row[1] if row[1] else 0
        
        if county not in county_stats:
            county_stats[county] = {"parcels_with_buildings": 0, "total_building_area": 0}
        
        if area > 0:
            county_stats[county]["parcels_with_buildings"] += 1
            county_stats[county]["total_building_area"] += area

print(f"\nBuilding coverage by county:")
for county, stats in county_stats.items():
    print(f"  {county}: {stats['parcels_with_buildings']:,} parcels with buildings")
    print(f"         {stats['total_building_area']:,.0f} sq ft total building area")

print(f"\n✅ Ready for disposal field analysis!")
print(f"✅ Building footprints available by county")
print(f"✅ Master table has Area_Under_Structures field")

=== COMPREHENSIVE BUILDING FOOTPRINTS BY COUNTY ===
Processing all counties with appropriate data sources

Step 1: Verify source layers and master table...
Available source layers:
  Honolulu: Building Footprints CCH ❌
  Maui: Maui County Building Footprints circa 2023 ❌
  Hawaii: USA_Structures_B ❌
  Kauai: USA_Structures_B ❌

Master table: 82,141 parcels ✅

Step 2: Processing each county...

--- Processing Honolulu County ---
❌ Source layer not available: Building Footprints CCH

--- Processing Maui County ---
❌ Source layer not available: Maui County Building Footprints circa 2023

--- Processing Hawaii County ---
❌ Source layer not available: USA_Structures_B

--- Processing Kauai County ---
❌ Source layer not available: USA_Structures_B

Step 3: Creating building area summary for master table...

=== BUILDING FOOTPRINTS PROCESSING COMPLETE ===
County results:

Master table updated:
  Total buildings processed: 0
  Field added: Area_Under_Structures

Building coverage by county:
  

In [46]:
print("=== IMPORTING BUILDING LAYERS TO GEODATABASE ===")
print("Step 1: Export hosted layers to local geodatabase")
print()

import arcpy
import os

# Define paths
gdb_path = r"C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\ParcelAnalysis.gdb"

# Get current map and layers
aprx = arcpy.mp.ArcGISProject("CURRENT")
map_obj = aprx.listMaps()[0]

# Define layer mappings for import
layer_imports = {
    "Building Footprints CCH": "Honolulu_Buildings_Raw",
    "Maui County Building Footprints circa 2023": "Maui_Buildings_Raw", 
    "USA_Structures_B": "USA_Structures_Raw"
}

print("Available layers in current map:")
available_layers = {}
for layer in map_obj.listLayers():
    print(f"  - {layer.name}")
    available_layers[layer.name] = layer

print(f"\nImporting building layers to geodatabase...")

imported_layers = {}

for source_name, target_name in layer_imports.items():
    if source_name in available_layers:
        print(f"\nImporting {source_name}...")
        
        try:
            source_layer = available_layers[source_name]
            target_path = os.path.join(gdb_path, target_name)
            
            # Remove existing if present
            if arcpy.Exists(target_path):
                arcpy.management.Delete(target_path)
                print(f"  Removed existing {target_name}")
            
            # Import the layer
            print(f"  Copying features to geodatabase...")
            arcpy.management.CopyFeatures(source_layer, target_path)
            
            # Verify import
            count = int(arcpy.management.GetCount(target_path)[0])
            print(f"  ✅ Imported {count:,} features to {target_name}")
            
            imported_layers[source_name] = {
                "target_name": target_name,
                "target_path": target_path,
                "count": count
            }
            
        except Exception as e:
            print(f"  ❌ Failed to import {source_name}: {str(e)}")
    else:
        print(f"\n❌ Layer not found: {source_name}")

print(f"\n=== IMPORT SUMMARY ===")
for source, info in imported_layers.items():
    print(f"✅ {source} → {info['target_name']} ({info['count']:,} features)")

if len(imported_layers) > 0:
    print(f"\n✅ Building layers successfully imported to geodatabase!")
    print(f"✅ Ready to run county-specific building extraction")
else:
    print(f"\n❌ No building layers imported - check layer names")

print(f"\nNext step: Run the county processing script with local geodatabase layers")

=== IMPORTING BUILDING LAYERS TO GEODATABASE ===
Step 1: Export hosted layers to local geodatabase

Available layers in current map:

Importing building layers to geodatabase...

❌ Layer not found: Building Footprints CCH

❌ Layer not found: Maui County Building Footprints circa 2023

❌ Layer not found: USA_Structures_B

=== IMPORT SUMMARY ===

❌ No building layers imported - check layer names

Next step: Run the county processing script with local geodatabase layers


In [47]:
print("=== FINDING BUILDING LAYERS IN MAP ===")
print("Scanning all layers to identify building footprint layers")
print()

import arcpy

# Get current map and list all layers
aprx = arcpy.mp.ArcGISProject("CURRENT")
map_obj = aprx.listMaps()[0]

print("All layers in current map:")
building_candidates = []

for i, layer in enumerate(map_obj.listLayers(), 1):
    layer_name = layer.name
    
    # Look for building-related keywords
    is_building_layer = any(keyword in layer_name.lower() for keyword in 
                           ['building', 'footprint', 'structure', 'usa_structures'])
    
    print(f"{i:2}. {layer_name} {'🏢' if is_building_layer else ''}")
    
    if is_building_layer:
        try:
            # Try to get basic info about the layer
            if hasattr(layer, 'dataSource'):
                print(f"     Source: {layer.dataSource}")
            
            # Try to get count (this might fail for some hosted layers)
            try:
                count = int(arcpy.management.GetCount(layer)[0])
                print(f"     Count: {count:,} features")
                
                building_candidates.append({
                    'name': layer_name,
                    'layer': layer,
                    'count': count
                })
            except:
                print(f"     Count: Unable to determine (hosted layer)")
                building_candidates.append({
                    'name': layer_name,
                    'layer': layer,
                    'count': 'Unknown'
                })
                
        except Exception as e:
            print(f"     Error getting info: {str(e)}")

print(f"\nBuilding layer candidates found:")
if building_candidates:
    for candidate in building_candidates:
        print(f"  - {candidate['name']} ({candidate['count']} features)")
else:
    print("  No building layers found")

print(f"\nPlease confirm the exact layer names for:")
print("1. Honolulu/Oahu building footprints")
print("2. Maui County building footprints") 
print("3. USA Structures layer")


=== FINDING BUILDING LAYERS IN MAP ===
Scanning all layers to identify building footprint layers

All layers in current map:

Building layer candidates found:
  No building layers found

Please confirm the exact layer names for:
1. Honolulu/Oahu building footprints
2. Maui County building footprints
3. USA Structures layer


In [48]:
print("=== IMPORTING BUILDING LAYERS TO GEODATABASE (FIXED) ===")
print("Using exact layer names from Contents pane")
print()

import arcpy
import os

# Define paths
gdb_path = r"C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\ParcelAnalysis.gdb"

# Get current map
aprx = arcpy.mp.ArcGISProject("CURRENT")
map_obj = aprx.listMaps()[0]

# Define exact layer names from your Contents pane
layer_imports = {
    "Building Footprints CCH": "Honolulu_Buildings_Raw",
    "Maui County Building Footprints circa 2023": "Maui_Buildings_Raw", 
    "USA_Structures_B": "USA_Structures_Raw"
}

print("Debug: Checking layer access method...")

# Try different methods to access layers
all_layers = map_obj.listLayers()
print(f"Total layers found: {len(all_layers)}")

for layer in all_layers:
    layer_name = layer.name
    print(f"  Found layer: '{layer_name}'")

print(f"\nAttempting imports...")

imported_layers = {}

for source_name, target_name in layer_imports.items():
    print(f"\nProcessing: {source_name}")
    
    # Find the layer by exact name match
    target_layer = None
    for layer in all_layers:
        if layer.name == source_name:
            target_layer = layer
            break
    
    if target_layer:
        print(f"  ✅ Found layer: {source_name}")
        
        try:
            target_path = os.path.join(gdb_path, target_name)
            
            # Remove existing if present
            if arcpy.Exists(target_path):
                arcpy.management.Delete(target_path)
                print(f"  Removed existing {target_name}")
            
            # Import the layer - try different approaches
            print(f"  Attempting to copy features...")
            
            # Method 1: Direct copy
            try:
                arcpy.management.CopyFeatures(target_layer, target_path)
                print(f"  Method 1 (direct copy) succeeded")
            except:
                print(f"  Method 1 failed, trying method 2...")
                # Method 2: Copy via layer reference
                arcpy.conversion.FeatureClassToFeatureClass(
                    target_layer, 
                    gdb_path, 
                    target_name
                )
                print(f"  Method 2 (conversion) succeeded")
            
            # Verify import
            count = int(arcpy.management.GetCount(target_path)[0])
            print(f"  ✅ Imported {count:,} features to {target_name}")
            
            imported_layers[source_name] = {
                "target_name": target_name,
                "target_path": target_path,
                "count": count
            }
            
        except Exception as e:
            print(f"  ❌ Failed to import {source_name}: {str(e)}")
            
            # Additional debug info
            try:
                print(f"     Layer type: {type(target_layer)}")
                print(f"     Layer data source: {target_layer.dataSource if hasattr(target_layer, 'dataSource') else 'No dataSource'}")
            except:
                print(f"     Could not get layer details")
    else:
        print(f"  ❌ Layer not found: {source_name}")

print(f"\n=== IMPORT SUMMARY ===")
for source, info in imported_layers.items():
    print(f"✅ {source} → {info['target_name']} ({info['count']:,} features)")

if len(imported_layers) > 0:
    print(f"\n✅ Building layers successfully imported to geodatabase!")
    print(f"✅ Ready to run county-specific building extraction")
    
    # Show what's now in the geodatabase
    print(f"\nVerifying imports in geodatabase:")
    arcpy.env.workspace = gdb_path
    feature_classes = arcpy.ListFeatureClasses("*Buildings_Raw")
    for fc in feature_classes:
        count = int(arcpy.management.GetCount(fc)[0])
        print(f"  {fc}: {count:,} features")
        
else:
    print(f"\n❌ No building layers imported - need to investigate layer access")

print(f"\nNext step: Run county processing with imported geodatabase layers")

=== IMPORTING BUILDING LAYERS TO GEODATABASE (FIXED) ===
Using exact layer names from Contents pane

Debug: Checking layer access method...
Total layers found: 0

Attempting imports...

Processing: Building Footprints CCH
  ❌ Layer not found: Building Footprints CCH

Processing: Maui County Building Footprints circa 2023
  ❌ Layer not found: Maui County Building Footprints circa 2023

Processing: USA_Structures_B
  ❌ Layer not found: USA_Structures_B

=== IMPORT SUMMARY ===

❌ No building layers imported - need to investigate layer access

Next step: Run county processing with imported geodatabase layers


In [1]:
print("=== EXTRACTING HAWAII BUILDINGS VIA PYTHON ===")
print("Using spatial intersection with Hawaii parcels")
print()

import arcpy
import os

# Define paths
gdb_path = r"C:\Users\rober\OneDrive\Documents\GIS_Projects\ParcelAnalysis\ParcelAnalysis.gdb"
master_table = os.path.join(gdb_path, "Complete_Master_Analysis_Table")

# Get the USA_Structures_B layer from your map
aprx = arcpy.mp.ArcGISProject("CURRENT")
map_obj = aprx.listMaps()[0]

usa_structures = None
for layer in map_obj.listLayers():
    if layer.name == "USA_Structures_B":
        usa_structures = layer
        break

if usa_structures:
    print("✅ Found USA_Structures_B layer")
    
    # Create Hawaii county boundary from master table
    print("Step 1: Creating Hawaii county boundary...")
    hawaii_parcels = os.path.join(gdb_path, "temp_hawaii_parcels")
    hawaii_boundary = os.path.join(gdb_path, "temp_hawaii_boundary")
    
    # Select Hawaii county parcels
    arcpy.management.Select(master_table, hawaii_parcels, "county = 'Hawaii'")
    hawaii_parcel_count = int(arcpy.management.GetCount(hawaii_parcels)[0])
    print(f"Hawaii parcels: {hawaii_parcel_count:,}")
    
    # Create boundary
    arcpy.management.MinimumBoundingGeometry(
        hawaii_parcels, 
        hawaii_boundary,
        "CONVEX_HULL",
        "ALL"
    )
    
    print("Step 2: Selecting buildings in Hawaii...")
    
    # Select buildings that intersect Hawaii boundary
    arcpy.management.SelectLayerByLocation(
        usa_structures,
        "INTERSECT",
        hawaii_boundary,
        selection_type="NEW_SELECTION"
    )
    
    selected_count = int(arcpy.management.GetCount(usa_structures)[0])
    print(f"Buildings selected: {selected_count:,}")
    
    if selected_count > 0:
        print("Step 3: Copying selected buildings...")
        
        hawaii_buildings = os.path.join(gdb_path, "Hawaii_Buildings_Raw")
        
        # Remove existing
        if arcpy.Exists(hawaii_buildings):
            arcpy.management.Delete(hawaii_buildings)
        
        # Copy selected features
        arcpy.management.CopyFeatures(usa_structures, hawaii_buildings)
        
        final_count = int(arcpy.management.GetCount(hawaii_buildings)[0])
        print(f"✅ Extracted {final_count:,} Hawaii building footprints")
        
        # Clear selection
        arcpy.management.SelectLayerByAttribute(usa_structures, "CLEAR_SELECTION")
        
        # Clean up temp files
        arcpy.management.Delete(hawaii_parcels)
        arcpy.management.Delete(hawaii_boundary)
        
        print("\n✅ Hawaii buildings ready!")
        print("Next: Extract Kauai buildings using same method")
        
    else:
        print("❌ No buildings found in Hawaii area")
        
else:
    print("❌ USA_Structures_B layer not found in map")
    

=== EXTRACTING HAWAII BUILDINGS VIA PYTHON ===
Using spatial intersection with Hawaii parcels

❌ USA_Structures_B layer not found in map
